## Data preparation

In [ ]:
import os
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
from pathlib import Path
import numpy as np
from sklearn.model_selection import train_test_split
import random
import pickle
import datetime
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

# DATASET_DIR = (Path("..") / ".." / "datasets").resolve()
# DATASETS = ["OFFICE-MANNERSDB", "MANNERSDBPlus"]
# LABEL_COLS = [
#     "Vaccum Cleaning", "Mopping the Floor", "Carry Warm Food",
#     "Carry Cold Food", "Carry Drinks", "Carry Small Objects",
#     "Carry Large Objects", "Cleaning", "Starting a conversation"
# ]

import sys
sys.path.append('..')

from data_processing.data_processing import ImageLabelDataset,DualImageDataset,create_dataloaders,create_crossvalidation_loaders

In [ ]:
df = pd.read_pickle("../data/pepper_data.pkl")

## CL Models

### imports

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm.notebook import tqdm, trange
import torch.optim as optim
from torchvision import models
import numpy as np
from collections import deque
import random
import time
import torch.nn.functional as F
from torch.utils.data import ConcatDataset, DataLoader, Subset
import torchvision.models.segmentation as segmentation
from collections import defaultdict

### ForgettingGatedSplitModel

In [ ]:
import os
import sys
import cv2
import pickle
import datetime
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from ultralytics import YOLO
from tqdm.notebook import tqdm, trange
from typing import Tuple, Optional
from collections import deque

sys.path.append('D:/projects/Depth-Anything-V2')
from depth_anything_v2.dpt import DepthAnythingV2

#### backbone

In [ ]:
class YOLO11FeatureExtractor(nn.Module):
    """Extract features from YOLO11-seg backbone (layers 0-8)"""
    def __init__(self, model_path='../models/yolo11n-seg.pt'):
        super().__init__()
        yolo = YOLO(model_path)

        # Extract layers 0-8 as discussed (before SPPF to preserve spatial granularity)
        self.backbone = nn.Sequential(*list(yolo.model.model[:9]))
        del yolo
        
        # Freeze parameters
        for param in self.backbone.parameters():
            param.requires_grad = False
        
    def forward(self, x):
        return self.backbone(x)  # Output: [B, 256, H/32, W/32]

class DepthAnythingFeatureExtractor(nn.Module):
    def __init__(self, model_path='../models/depth_anything_v2_vits.pth'):
        super().__init__()
        model_configs = {
            'vits': {'encoder': 'vits', 'features': 64, 'out_channels': [48, 96, 192, 384]},
        }
        model = DepthAnythingV2(**model_configs['vits'])
        model.load_state_dict(torch.load(model_path, map_location='cpu'))
        
        self.backbone = model.pretrained
        del model

        for param in self.backbone.parameters():
            param.requires_grad = False
    
    def forward(self, x):
        B, C, H, W = x.shape
        
        with torch.no_grad():
            # Patch embedding
            x = self.backbone.patch_embed(x)
            
            # Pass through transformer blocks
            for block in self.backbone.blocks:
                x = block(x)
            
            # Apply final norm
            x = self.backbone.norm(x)
            
            # Remove CLS token and reshape to spatial format
            if x.shape[1] > (H//14) * (W//14):
                patch_tokens = x[:, 1:, :]  # Remove CLS token
            else:
                patch_tokens = x
            
            # Reshape to spatial format
            patch_h, patch_w = H // 14, W // 14
            spatial_features = patch_tokens.transpose(1, 2).reshape(B, 384, patch_h, patch_w)
            
            return spatial_features


class DualBackboneFeatureExtractor(nn.Module):
    """Combines YOLO11 and DepthAnything feature extractors"""
    def __init__(self, model_paths):
        super().__init__()
        # Individual feature extractors
        self.yolo_extractor = YOLO11FeatureExtractor(model_paths['YOLO11n_seg'])
        self.depth_extractor = DepthAnythingFeatureExtractor(model_paths['DepthAnythingV2_small'])
        
        # Feature projection layers (optional - can keep features as-is)
        self.yolo_proj = nn.Conv2d(256, 256, 1)
        self.depth_proj = nn.Conv2d(384, 384, 1)
        
    def forward(self, x):
        # Extract features from both backbones
        yolo_features = self.yolo_extractor(x[0])  # [B, 256, H/32, W/32]
        depth_features = self.depth_extractor(x[1])  # [B, 384, H/14, W/14]
        
        # Apply projections
        yolo_features = self.yolo_proj(yolo_features)
        depth_features = self.depth_proj(depth_features)
        
        # Downsample the depthanything features to match yolo
        depth_features = F.adaptive_avg_pool2d(depth_features, (7, 7)) #image size /32 224,224 > 7,7
        
        # Concatenate features
        combined_features = torch.cat([yolo_features, depth_features], dim=1)  # [B, 640, H/14, W/14] 
        
        return combined_features

#### model components

In [ ]:
class CatastrophicForgettingAdapter(nn.Module):
    """Standalone adapter that measures forgetting and gates spatial features"""
    def __init__(self, num_channels=640, num_outputs=9):
        super().__init__()
        self.num_channels = num_channels
        
        # Forgetting measurement storage (per channel)
        self.previous_adapter_state = None
        self.current_forgetting_scores = None
        
        # Adapter that predicts main task (for forgetting measurement)
        self.adapter = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),  # Global average pooling
            nn.Flatten(),
            nn.Linear(num_channels, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, num_outputs)  # Predicts main task
        )
        
        # Learnable gating network (operates on channel-wise forgetting scores)
        self.gating = nn.Sequential(
            nn.Linear(num_channels, num_channels // 4),
            nn.ReLU(),
            nn.Linear(num_channels // 4, num_channels),
            nn.Sigmoid()
        )
        
    def forward(self, backbone_features, use_gating=True):
        """
        Args:
            backbone_features: [B, 640, 36, 36] spatial features
        Returns:
            forgetting_features: [B, 640, 36, 36] features routed to forgetting branch
            not_forgetting_features: [B, 640, 36, 36] features routed to stable branch
            adapter_predictions: [B, 9] task predictions for forgetting measurement
        """
        B, C, H, W = backbone_features.shape
        
        # Adapter makes predictions (for forgetting measurement)
        adapter_predictions = self.adapter(backbone_features)
        
        if use_gating and self.current_forgetting_scores is not None:
            # Apply channel-wise gating based on forgetting scores
            forgetting_scores_batch = self.current_forgetting_scores.unsqueeze(0).expand(B, -1)  # [B, 640]
            gating_mask = self.gating(forgetting_scores_batch)  # [B, 640]
            
            # Reshape for spatial broadcasting
            gating_mask = gating_mask.unsqueeze(2).unsqueeze(3)  # [B, 640, 1, 1]
            
            # Route features channel-wise
            forgetting_features = backbone_features * gating_mask
            not_forgetting_features = backbone_features * (1 - gating_mask)
        else:
            # No gating - both branches get all features
            forgetting_features = backbone_features
            not_forgetting_features = backbone_features
        
        return forgetting_features, not_forgetting_features, adapter_predictions
    
    def store_adapter_state(self):
        """Store current adapter state for forgetting measurement"""
        self.previous_adapter_state = {
            name: param.clone().detach() 
            for name, param in self.adapter.named_parameters()
        }
    
    def compute_forgetting_scores(self):
        """Compute channel-wise forgetting scores by comparing adapter states"""
        # Get the device from model parameters
        device = next(self.parameters()).device
        
        if self.previous_adapter_state is None:
            # First domain - no forgetting to measure
            self.current_forgetting_scores = torch.zeros(self.num_channels, device=device)
            return
        
        # Compare adapter weights to measure which channels are most affected
        forgetting_scores = []
        
        for name, current_param in self.adapter.named_parameters():
            if name in self.previous_adapter_state:
                previous_param = self.previous_adapter_state[name]
                
                if 'weight' in name and len(current_param.shape) == 2:
                    # For first linear layer after pooling: [256, 640]
                    if current_param.shape[1] == self.num_channels:
                        # Compute change per input channel
                        weight_change = torch.norm(current_param - previous_param, dim=0)
                        forgetting_scores.append(weight_change)
                        break  # Use only first layer that maps from channels
        
        # Use channel-wise forgetting scores
        if forgetting_scores:
            self.current_forgetting_scores = forgetting_scores[0].to(device)
            
            # Normalize to [0, 1]
            if self.current_forgetting_scores.max() > 0:
                self.current_forgetting_scores = (
                    self.current_forgetting_scores / self.current_forgetting_scores.max()
                )
        else:
            # Fallback if no forgetting scores computed
            self.current_forgetting_scores = torch.zeros(self.num_channels, device=device)

class ConvBranch(nn.Module):
    """Convolutional processing branch for spatial features"""
    def __init__(self, input_channels=640, hidden_channels=256, output_channels=128):
        super().__init__()
        self.conv_layers = nn.Sequential(
            # First conv block
            nn.Conv2d(input_channels, hidden_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(hidden_channels),
            nn.ReLU(inplace=True),
            nn.Dropout2d(0.1),
            
            # Second conv block
            nn.Conv2d(hidden_channels, hidden_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(hidden_channels),
            nn.ReLU(inplace=True),
            nn.Dropout2d(0.1),
            
            # Output conv block
            nn.Conv2d(hidden_channels, output_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(output_channels),
            nn.ReLU(inplace=True),
            
            # Global pooling to get fixed-size output
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten()  # [B, output_channels]
        )
        
    def forward(self, x):
        return self.conv_layers(x)
    
    
class FeatureFusion(nn.Module):
    """Attention-based fusion of branch features"""
    def __init__(self, feature_dim=256):
        super().__init__()
        
        self.attention = nn.Sequential(
            nn.Linear(feature_dim * 2, feature_dim),
            nn.ReLU(),
            nn.Linear(feature_dim, 2),
            nn.Softmax(dim=1)
        )
        
    def forward(self, high_features, low_features):
        # Concatenate features
        combined = torch.cat([high_features, low_features], dim=1)  # [B, 512]
        
        # Compute attention weights
        attention_weights = self.attention(combined)  # [B, 2]
        
        # Apply attention
        weighted_high = high_features * attention_weights[:, 0:1]
        weighted_low = low_features * attention_weights[:, 1:2]
        
        # Combine with residual connection
        fused = weighted_high + weighted_low
        
        return fused


#### model

In [ ]:
class CatastrophicForgettingDisentanglementModel(nn.Module):
    """Updated model with spatial features and convolutional branches"""
    
    def __init__(self, 
                 backbone_output_channels: int = 640,
                 branch_hidden_channels: int = 256,
                 branch_output_channels: int = 128,
                 num_outputs: int = 9):
        super().__init__()
        
        print("🚀 Initializing Spatial Catastrophic Forgetting Model...")
        
        # Dual backbone (frozen) - now outputs spatial features
        model_paths={
            'YOLO11n_seg': '../models/yolo11n-seg.pt', 
            'DepthAnythingV2_small': '../models/depth_anything_v2_vits.pth'
            }
        missing = [name for name, path in model_paths.items() if not os.path.exists(path)]
        if missing:
            raise FileNotFoundError(f"Missing model files for: {', '.join(missing)}")

        self.backbone = DualBackboneFeatureExtractor(model_paths=model_paths)  # [B, 640, 36, 36]
        
        # Standalone forgetting adapter
        self.forgetting_adapter = CatastrophicForgettingAdapter(
            num_channels=backbone_output_channels,
            num_outputs=num_outputs
        )
        
        # Convolutional branches for spatial processing
        self.branch_forgetting = ConvBranch(
            backbone_output_channels, 
            branch_hidden_channels, 
            branch_output_channels
        )
        self.branch_not_forgetting = ConvBranch(
            backbone_output_channels, 
            branch_hidden_channels, 
            branch_output_channels
        )
        
        self.fusion = FeatureFusion(branch_output_channels)

        # Final head combines branch outputs
        self.head =  nn.Sequential(
            nn.Linear(branch_output_channels, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.3),
            
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.2),
            
            nn.Linear(256, num_outputs)
        )
        
        print("✅ Spatial model initialized successfully!")
        
    def forward(self, x, use_gating: bool = True):
        # Extract spatial features from frozen backbone
        backbone_features = self.backbone(x)  # [B, 640, 36, 36]
        
        # Process through forgetting adapter (gating + task prediction)
        forgetting_features, not_forgetting_features, adapter_predictions = \
            self.forgetting_adapter(backbone_features, use_gating)
        
        # Process through convolutional branches
        forgetting_output = self.branch_forgetting(forgetting_features)      # [B, 128]
        not_forgetting_output = self.branch_not_forgetting(not_forgetting_features)  # [B, 128]
        
        # Combine branch outputs for final prediction
        fused_features = self.fusion(forgetting_output, not_forgetting_output)
        final_output = self.head(fused_features)
        
        return {
            'output': final_output,
            'adapter_output': adapter_predictions,
            'backbone_features': backbone_features,
            'forgetting_features': forgetting_features,
            'not_forgetting_features': not_forgetting_features
        }
    
    def store_adapter_state(self):
        """Delegate to forgetting adapter"""
        self.forgetting_adapter.store_adapter_state()
    
    def compute_forgetting_scores(self):
        """Delegate to forgetting adapter"""
        self.forgetting_adapter.compute_forgetting_scores()

    @property
    def current_forgetting_scores(self):
        """Access forgetting scores from adapter"""
        return self.forgetting_adapter.current_forgetting_scores


#### testing

In [ ]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # Initialize your model
# cf_model = CatastrophicForgettingDisentanglementModel(
#     backbone_output_channels=640,
#     branch_hidden_channels=256,
#     branch_output_channels=128,
#     num_outputs=9
# ).to(device)

# # Set model to evaluation mode (important for testing)
# cf_model.eval()

# # Create dummy input tensor matching your expected input size
# # Your model expects: [batch_size, 3, height, width]
# # Using 224x224 as we discussed for memory efficiency
# batch_size = 2
# dummy1 = torch.randn(batch_size, 3, 224, 224).to(device)
# dummy2 = torch.randn(batch_size, 3, 224, 224).to(device)
# dummy_input = (dummy1, dummy2)

# print(f"Input shape: {dummy_input[0].shape}")

# # Test forward pass without gating (first domain)
# with torch.no_grad():
#     outputs_no_gating = cf_model(dummy_input, use_gating=False)

# print("✅ Forward pass without gating successful!")
# print(f"Output shape: {outputs_no_gating['output'].shape}")
# print(f"Adapter output shape: {outputs_no_gating['adapter_output'].shape}")
# print(f"Backbone features shape: {outputs_no_gating['backbone_features'].shape}")

# # Test forward pass with gating (after first domain)
# # First, simulate having forgetting scores
# cf_model.forgetting_adapter.current_forgetting_scores = torch.randn(640).to(device)

# with torch.no_grad():
#     outputs_with_gating = cf_model(dummy_input, use_gating=True)

# print("✅ Forward pass with gating successful!")
# print(f"Forgetting features shape: {outputs_with_gating['forgetting_features'].shape}")
# print(f"Not forgetting features shape: {outputs_with_gating['not_forgetting_features'].shape}")

# # Test forgetting measurement functions
# cf_model.store_adapter_state()
# print("✅ Adapter state stored successfully!")

# cf_model.compute_forgetting_scores()
# print("✅ Forgetting scores computed successfully!")
# print(f"Forgetting scores shape: {cf_model.current_forgetting_scores.shape}")

# outputs = cf_model(dummy_input, use_gating=True)
# print(outputs['output'].device)

#### helper function

In [ ]:
def evaluate_model_cf(model, dataloader, criterion, device):
    """Modified evaluation function for catastrophic forgetting model"""
    model.eval()
    total_loss = 0.0
    total_samples = 0
    with torch.no_grad():
        for yolo_images, depth_images, labels, _ in dataloader:
            yolo_images = yolo_images.to(device, dtype=torch.float32)
            depth_images = depth_images.to(device, dtype=torch.float32)
            inputs = (yolo_images, depth_images)
            labels = labels.to(device, dtype=torch.float32)
            outputs = model(inputs, use_gating=True)  # Use gating during evaluation
            loss = criterion(outputs['output'], labels)
            total_loss += loss.item() * inputs[0].size(0)
            total_samples += inputs[0].size(0)
    return total_loss / total_samples

def cross_domain_validation_cf(model, domain_dataloaders, criterion, device):
    """Modified cross-domain validation for catastrophic forgetting model"""
    results = {}
    for domain, loaders in domain_dataloaders.items():
        val_loader = loaders['val']
        val_loss = evaluate_model_cf(model, val_loader, criterion, device)
        results[domain] = val_loss
    return results

def average_metrics(metrics_list):
    if not metrics_list:
        return {}
    keys = metrics_list[0].keys()
    avg_metrics = {}
    for k in keys:
        avg_metrics[k] = float(np.mean([m[k] for m in metrics_list if k in m]))
    return avg_metrics

#### training

In [ ]:
def compute_gating_loss(forgetting_scores: torch.Tensor, 
                       gating_mask: torch.Tensor,
                       lambda_balance: float = 1.0) -> torch.Tensor:
    """Compute loss for gating network"""
    high_forgetting_features = forgetting_scores * gating_mask
    low_forgetting_features = forgetting_scores * (1 - gating_mask)
    
    high_mean = high_forgetting_features.sum() / (gating_mask.sum() + 1e-8)
    low_mean = low_forgetting_features.sum() / ((1 - gating_mask).sum() + 1e-8)
    
    split_loss = -(high_mean - low_mean)
    balance_loss = torch.abs(gating_mask.mean() - 0.5)
    
    total_loss = split_loss + lambda_balance * balance_loss
    return total_loss

def catastrophic_forgetting_batch(model, batch, device, loss_params={'main': 1.0, 'adapter': 0.5, 'gating': 0.1}, **kwargs):
    """CORRECTED: Batch function with proper adapter access"""
    yolo_images, depth_images, labels, domain_labels = batch
    yolo_images = yolo_images.to(device)
    depth_images = depth_images.to(device)
    inputs = (yolo_images, depth_images)
    labels = labels.to(device)
    
    mse_criterion = kwargs['mse_criterion']
    domain_idx = kwargs.get('domain_idx', 0)
    
    # Forward pass
    outputs = model(inputs, use_gating=(domain_idx > 0))
    
    # Main task loss (from final head)
    main_loss = mse_criterion(outputs['output'], labels)
    
    # Adapter loss (for monitoring forgetting)
    adapter_loss = mse_criterion(outputs['adapter_output'], labels)
    
    # Main loss backpropagation
    main_loss.backward(retain_graph=True) 
    
    # CORRECTED: Gating loss with proper path access
    gating_loss = torch.tensor(0.0, device=device)
    if domain_idx > 0 and model.current_forgetting_scores is not None:
        backbone_features = outputs['backbone_features']
        batch_size = backbone_features.size(0)
        forgetting_scores_batch = model.current_forgetting_scores.unsqueeze(0).expand(batch_size, -1).to(device)
        
        # FIXED: Use correct path to gating network
        gating_mask = model.forgetting_adapter.gating(forgetting_scores_batch)
        
        gating_loss = compute_gating_loss(
            model.current_forgetting_scores.to(device), 
            gating_mask.mean(dim=0)
        )
    
    metrics = {
        'main_loss': main_loss.item(),
        'adapter_loss': adapter_loss.item(),
        'gating_loss': gating_loss.item()
    }
    
    return main_loss, gating_loss, metrics

def catastrophic_forgetting_train_loop(
    model, domains, domain_dataloaders, buffer, optimizer, gating_optimizer, device,
    batch_fn, batch_kwargs, loss_params, num_epochs=5, exp_name="cf_exp", 
    gradient_clipping=False, restart={}
):
    """CORRECTED: Training loop with proper forgetting measurement"""
    start_domain_idx = 0
    global_step = 0
    history = {
        'train_epoch_loss': [],
        'val_epoch_loss': [],
        'train_epoch_metrics': [],
        'cross_domain_val': [],
        'grad_norms': [],
        'forgetting_scores_history': [],
        'gating_losses': []
    }
    
    if restart:
        global_step = restart['global_step']
        history = restart['history']
        start_domain_idx = np.where(domains == restart['domain'])[0][0]
        for domain_idx, current_domain in enumerate(domains[:start_domain_idx]):
            buffer.update_buffer(current_domain, domain_dataloaders[current_domain]['train'].dataset) 
        print(f"Restarting from domain {restart['domain']} index {start_domain_idx}")
        print(f"Buffer: {buffer.get_domain_distribution()}")         

    for domain_idx, current_domain in enumerate(tqdm(domains[start_domain_idx:], desc=f"Total training"), start=start_domain_idx):
        print(f"\n=== Training on Domain {domain_idx}: {current_domain} ===")
        
        # Store adapter state before training (for forgetting measurement)
        if domain_idx > 0:
            model.store_adapter_state()
        
        train_loader = buffer.get_loader_with_replay(current_domain, domain_dataloaders[current_domain]['train'])
        
        for epoch in trange(num_epochs, desc=f"Current domain {current_domain}"):
            model.train()
            epoch_loss = 0.0
            samples = 0
            batch_metrics_list = []
            
            for batch_idx, batch in enumerate(tqdm(train_loader, desc=f"Current epoch {epoch}", leave=False)):
                # Main model training
                optimizer.zero_grad()
                
                batch_kwargs_with_domain = {**batch_kwargs, 'current_domain': current_domain, 'domain_idx': domain_idx}
                
                main_loss, gating_loss, metrics = batch_fn(model, batch, device, loss_params, **batch_kwargs_with_domain)
                
                if gradient_clipping:
                    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                optimizer.step()
                
                # Gating training (in same batch, separate optimizer)
                if domain_idx > 0 and gating_loss.item() > 0:
                    gating_optimizer.zero_grad()
                    gating_loss.backward()
                    gating_optimizer.step()
                
                batch_size = batch[0].size(0)
                epoch_loss += main_loss.item() * batch_size
                samples += batch_size
                global_step += 1
                batch_metrics_list.append(metrics)
                
            avg_epoch_loss = epoch_loss / samples
            history['train_epoch_loss'].append(avg_epoch_loss)
            
            avg_metrics = average_metrics(batch_metrics_list)
            history['train_epoch_metrics'].append(avg_metrics)
            
            grad_norms = collect_gradients(model)
            history['grad_norms'].append(grad_norms)
            
            # Validation on current domain
            val_loss = evaluate_model_cf(model, domain_dataloaders[current_domain]['val'], batch_kwargs['mse_criterion'], device)
            history['val_epoch_loss'].append(val_loss)
            
            # Cross-domain validation (after each domain)
            if epoch == num_epochs-1:
                cross_val = cross_domain_validation_cf(model, domain_dataloaders, batch_kwargs['mse_criterion'], device)
                history['cross_domain_val'].append(cross_val)

                # Compute forgetting scores after training on domain (for next domain)
                model.compute_forgetting_scores()
                history['forgetting_scores_history'].append(
                    model.current_forgetting_scores.clone() if model.current_forgetting_scores is not None else None
                )
                if model.current_forgetting_scores is not None:
                    print(f"Forgetting scores computed. Mean: {model.current_forgetting_scores.mean():.4f}, "
                        f"Std: {model.current_forgetting_scores.std():.4f}")
                

                # Save checkpoint
                torch.save({
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'gating_optimizer_state_dict': gating_optimizer.state_dict(),
                    'history': history,
                    'forgetting_scores': model.current_forgetting_scores,
                }, f"../checkpoints/{exp_name}_domain{current_domain}_epoch{epoch}_step{global_step}.pt")
            
            with open(f"../checkpoints/{exp_name}_history.pkl", "wb") as f:
                pickle.dump(history, f)
        
        buffer.update_buffer(current_domain, domain_dataloaders[current_domain]['train'].dataset)
        print(f"Domain {domain_idx} completed. Buffer: {buffer.get_domain_distribution()}")
    
    return history


### Explicit Heuristic Split Model

#### ZoeDepth - HuggingFace

In [ ]:
# import os
# from pathlib import Path
# import torch
# import numpy as np
# from transformers import AutoImageProcessor, ZoeDepthForDepthEstimation
# from PIL import Image
# from tqdm import tqdm
# from torchvision import transforms

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# output_dir = Path('../data/depth')
# output_dir.mkdir(parents=True, exist_ok=True)

# image_processor = AutoImageProcessor.from_pretrained("Intel/zoedepth-nyu-kitti", use_fast=True)
# model = ZoeDepthForDepthEstimation.from_pretrained("Intel/zoedepth-nyu-kitti").to(device).eval()

# # Prepare image paths list
# image_paths = df['image_path'].tolist()

# batch_size = 10  # or whatever batch size you want
# for batch_idx in tqdm(range(0, len(image_paths), batch_size)):
#     batch_paths = image_paths[batch_idx:batch_idx + batch_size]
    
#     # Load images as PIL Images (no manual transform)
#     batch_images = [Image.open(img_path).convert("RGB") for img_path in batch_paths]
    
#     # Preprocess with ZoeDepth image processor
#     inputs = image_processor(images=batch_images, return_tensors="pt").to(device)
    
#     with torch.no_grad():
#         outputs = model(**inputs)
    
#     # Post-process depth maps to original sizes
#     source_sizes = [(img.height, img.width) for img in batch_images]
#     post_processed = image_processor.post_process_depth_estimation(
#         outputs,
#         source_sizes=source_sizes
#     )
    
#     for i, depth_dict in enumerate(post_processed):
#         # Get raw depth map
#         depth_array = depth_dict["predicted_depth"].cpu().numpy()
#         img_stem = Path(batch_paths[i]).stem
#         np.save(output_dir / f"{img_stem}.npy", depth_array)
#         # Save visualization PNG
#         depth_norm = (depth_array - depth_array.min()) / (depth_array.max() - depth_array.min())
#         depth_img = Image.fromarray((depth_norm * 255).astype(np.uint8))
#         depth_img.save(output_dir / f"{img_stem}_depth.png")



#### segmentation generation

In [ ]:
from autodistill_grounded_sam import GroundedSAM
from autodistill.detection import CaptionOntology
from autodistill.utils import plot
import cv2
import pickle
import bz2


import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
# define an ontology to map class names to our GroundedSAM prompt
# the ontology dictionary has the format {caption: class}
# where caption is the prompt sent to the base model, and class is the label that will
# be saved for that caption in the generated annotations
# then, load the model
base_model = GroundedSAM(
    ontology=CaptionOntology(
        {
            "human . child . person": "human",
            "robot": "robot",
            "dog": "dog"
        }
    )
)

# human : human, anima: animal, robot:robot

In [ ]:
# # run inference on a single image
# results = base_model.predict(df['image_path'].iloc[0])

# plot(
#     image=cv2.imread(df['image_path'].iloc[0]),
#     classes=base_model.ontology.classes(),
#     detections=results
# )


In [ ]:
with bz2.BZ2File('autodistill_dataset_home.pbz2', 'rb') as f:
    dataset_home = pickle.load(f)

In [ ]:
# with bz2.BZ2File('autodistill_dataset.pbz2', 'wb') as f:
#     pickle.dump(dataset, f, protocol=pickle.HIGHEST_PROTOCOL)


In [ ]:

gc.collect()
torch.cuda.empty_cache()

In [ ]:
# dataset_home = base_model.label("../../socialsense/data/images/home", extension=".png")
# with bz2.BZ2File('autodistill_dataset_home.pbz2', 'wb') as f:
#     pickle.dump(dataset_home, f, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open("autodistill_temp_dataset.pkl", "rb") as f:
    dataset = pickle.load(f)

In [ ]:
from PIL import Image, ImageDraw, ImageFont
import numpy as np
import os
import supervision as sv
from tqdm.notebook import tqdm, trange

# Define a color palette for your classes
# Use RGBA for masks (with transparency), RGB for boxes (solid)
class_colors = {
    0: (255, 0, 0, 100),    # Red for class 0 (human)
    1: (0, 255, 0, 100),    # Green for class 1 (robot)
    2: (0, 0, 255, 100),    # Blue for class 2 (animal)
    3: (122, 122, 0, 100),
    # Add more if you have more classes
}

def visualize_and_save_pil_colored(dataset, confidence_threshold=0.3, output_dir="../data/temp_masks_coloured"):
    os.makedirs(output_dir, exist_ok=True)
    
    for idx, (path, image, detections) in enumerate(tqdm(dataset)):
        # Convert BGR to RGB for PIL (if your image is BGR)
        rgb_image = image[:, :, ::-1]
        pil_img = Image.fromarray(rgb_image)
        draw = ImageDraw.Draw(pil_img, "RGBA")
        
        # Filter detections by confidence
        keep_indices = [i for i, conf in enumerate(detections.confidence) if conf >= confidence_threshold]
        if not keep_indices:
            print(f"No detections above threshold for image {path}")
            continue

        filtered_boxes = detections.xyxy[keep_indices]
        filtered_masks = detections.mask[keep_indices]
        filtered_confidences = detections.confidence[keep_indices]
        filtered_class_ids = detections.class_id[keep_indices]
        
        # Overlay masks with transparency and class colors
        for i, (mask, conf) in enumerate(zip(filtered_masks, filtered_confidences)):
            class_id = filtered_class_ids[i] if filtered_class_ids is not None else 0
            color = class_colors.get(class_id, (255, 255, 255, 100))  # Default white if class unknown
            # Create a colored mask with alpha
            mask_img = Image.fromarray((mask * 255).astype(np.uint8), mode="L")
            colored_mask = Image.new("RGBA", pil_img.size, color)
            # Composite the colored mask onto the image with transparency
            pil_img = Image.alpha_composite(pil_img.convert("RGBA"), Image.composite(colored_mask, Image.new("RGBA", pil_img.size), mask_img))
        
        draw = ImageDraw.Draw(pil_img)
        
        # Draw bounding boxes and confidence with class colors
        for i, (box, conf) in enumerate(zip(filtered_boxes, filtered_confidences)):
            class_id = filtered_class_ids[i] if filtered_class_ids is not None else 0
            color = class_colors[class_id][:3]  # Use RGB for boxes (no alpha)
            x1, y1, x2, y2 = map(int, box)
            draw.rectangle([x1, y1, x2, y2], outline=color, width=2)
            draw.text((x1, y1 - 10*class_id), f"{class_id}: {conf:.2f}", fill=color)
        
        # Save the image as PNG
        filename = os.path.basename(path)
        save_path = os.path.join(output_dir, os.path.splitext(filename)[0] + ".png")
        pil_img.convert("RGB").save(save_path)


# Example usage



In [ ]:
import numpy as np
from supervision import Detections

def calculate_iou(box1, box2):
    """Calculate Intersection over Union for two boxes [x1,y1,x2,y2]"""
    x1_inter = max(box1[0], box2[0])
    y1_inter = max(box1[1], box2[1])
    x2_inter = min(box1[2], box2[2])
    y2_inter = min(box1[3], box2[3])
    inter_area = max(0, x2_inter - x1_inter) * max(0, y2_inter - y1_inter)
    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])
    return inter_area / (box1_area + box2_area - inter_area + 1e-6)

def remove_duplicates_any_class(detections: Detections, iou_threshold=0.9) -> Detections:
    """
    Remove duplicate detections using IoU and confidence, regardless of class.
    
    Args:
        detections: supervision.Detections object
        iou_threshold: IoU threshold for considering duplicates
        
    Returns:
        Filtered Detections object
    """
    # Extract components from Detections
    xyxy = detections.xyxy
    confidence = detections.confidence
    class_id = detections.class_id
    mask = detections.mask

    # Convert to list of dicts for processing
    detections_list = [
        {
            'box': xyxy[i],
            'mask': mask[i] if mask is not None else None,
            'confidence': confidence[i],
            'class_id': class_id[i]
        }
        for i in range(len(xyxy))
    ]

    # Sort by confidence (highest first)
    detections_list.sort(key=lambda x: x['confidence'], reverse=True)
    
    # Filter duplicates (do NOT check class_id)
    keep = []
    while detections_list:
        current = detections_list.pop(0)
        keep.append(current)
        detections_list = [
            d for d in detections_list
            if calculate_iou(current['box'], d['box']) <= iou_threshold
        ]

    # Reconstruct Detections object
    return Detections(
        xyxy=np.array([d['box'] for d in keep]),
        confidence=np.array([d['confidence'] for d in keep]),
        class_id=np.array([d['class_id'] for d in keep]),
        mask=np.array([d['mask'] for d in keep]) if mask is not None else None
    )

# Usage example


In [ ]:
filtered_dataset = []
for path, image, detections in dataset_746:
    filtered_detections = remove_duplicates_any_class(detections, iou_threshold=0.95)
    filtered_dataset.append((path, image, filtered_detections))

In [ ]:
# visualize_and_save_pil_colored(dataset, confidence_threshold=0.3)

#### depth mean std

In [ ]:
import numpy as np
import os
from tqdm.notebook import tqdm

def calculate_mean_std_for_npy(folder_path):
    total_sum = 0
    total_sum_sq = 0
    total_count = 0

    # List all .npy files
    files = [f for f in os.listdir(folder_path) if f.endswith('.npy')]
    
    # Wrap the loop with tqdm for progress bar
    for filename in tqdm(files):
        file_path = os.path.join(folder_path, filename)
        img = np.load(file_path).astype(np.float64)
        total_sum += img.sum()
        total_sum_sq += (img ** 2).sum()
        total_count += img.size

    mean = total_sum / total_count if total_count > 0 else None
    variance = (total_sum_sq / total_count) - (mean ** 2) if total_count > 0 else None
    std = np.sqrt(variance) if variance is not None else None
    return mean, std


In [ ]:
# calculate_mean_std_for_npy('../data/depth')

#### mask fusion

In [ ]:
import cv2
import numpy as np
import torch
import torchvision.transforms as transforms
from PIL import Image
import os
from pathlib import Path

def process_segmentation_data(detections_dataset, output_dir, imagenet_mean=[0.485, 0.456, 0.406], confidence_threshold=0.3):
    """
    Process supervision Detections dataset to create social and environment images
    
    Args:
        detections_dataset: List of tuples (image_path, Detection_object, ...)
        output_dir: Base directory to save processed images
        imagenet_mean: RGB mean values for filling masked areas
    """
    
    # Create output directories
    social_dir = Path(output_dir) / "social"
    env_dir = Path(output_dir) / "environment"
    social_dir.mkdir(parents=True, exist_ok=True)
    env_dir.mkdir(parents=True, exist_ok=True)
    
    for item in tqdm(detections_dataset):
        image_path = item[0]
        detections = item[2]
        
        # Load original image
        original_image = cv2.imread(str(image_path))
        original_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)
        height, width = original_image.shape[:2]
        
        # Get filename for saving
        filename = Path(image_path).name

        keep_indices = [i for i, conf in enumerate(detections.confidence) if conf >= confidence_threshold]
        filtered_masks = detections.mask[keep_indices]
        
        # Combine all masks into one
        combined_mask = combine_masks(filtered_masks, height, width)
        
        # Create social image (only people and robot visible)
        social_image = apply_mask_with_mean(
            original_image, combined_mask, imagenet_mean, keep_masked=True
        )
        
        # Create environment image (room only, people and robot masked out)
        env_image = apply_mask_with_mean(
            original_image, combined_mask, imagenet_mean, keep_masked=False
        )
        
        # Save images
        social_path = social_dir / filename
        env_path = env_dir / filename
        
        save_image(social_image, social_path)
        save_image(env_image, env_path)


def combine_masks(masks, height, width, confidence_threshold=0.3):
    """
    Combine multiple masks into a single binary mask using union operation
    
    Args:
        masks: Array of individual masks from Detection object
        height, width: Dimensions of the original image
        
    Returns:
        combined_mask: Single binary mask (1 = object, 0 = background)
    """
    if masks is None or len(masks) == 0:
        return np.zeros((height, width), dtype=np.uint8)
    
    # Initialize combined mask
    combined_mask = np.zeros((height, width), dtype=np.uint8)
    
    # Union all individual masks using maximum function (as shown in search results)
    for mask in masks:
        # Ensure mask is the right size
        if mask.shape != (height, width):
            raise ValueError(f"Mask shape incorrect: {mask.shape}, should be {(height, width)}")
        
        # Union operation: take maximum of current combined mask and new mask
        combined_mask = np.maximum(combined_mask, mask.astype(np.uint8))
    
    return combined_mask

def apply_mask_with_mean(image, mask, imagenet_mean, keep_masked=True):
    """
    Apply mask to image and fill empty areas with ImageNet mean values
    
    Args:
        image: Original RGB image (H, W, 3)
        mask: Binary mask (H, W) where 1 = object, 0 = background
        imagenet_mean: RGB mean values [R, G, B] in range [0, 1]
        keep_masked: If True, keep masked areas (social). If False, remove masked areas (environment)
        
    Returns:
        processed_image: Image with mask applied and filled with mean values
    """
    processed_image = image.copy().astype(np.float32) / 255.0
    
    # Convert imagenet_mean to same range as image
    mean_values = np.array(imagenet_mean).reshape(1, 1, 3)
    
    if keep_masked:
        # Social image: keep people/robot, fill background with mean
        fill_mask = (mask == 0)  # Areas to fill (background)
    else:
        # Environment image: keep background, fill people/robot with mean  
        fill_mask = (mask == 1)  # Areas to fill (people/robot)
    
    # Fill specified areas with ImageNet mean values
    for c in range(3):  # RGB channels
        processed_image[:, :, c][fill_mask] = imagenet_mean[c]
    
    # Convert back to uint8
    processed_image = (processed_image * 255).astype(np.uint8)
    
    return processed_image

def save_image(image, save_path):
    """
    Save image to specified path
    
    Args:
        image: RGB image array (H, W, 3)
        save_path: Path to save the image
    """
    # Convert to PIL Image and save
    pil_image = Image.fromarray(image)
    pil_image.save(save_path)


In [ ]:
# with bz2.BZ2File('autodistill_dataset_home.pbz2', 'rb') as f:
#     dataset_home = pickle.load(f)

# process_segmentation_data(
#     detections_dataset=dataset_home,
#     output_dir='../data/masked',
#     imagenet_mean=[0.485, 0.456, 0.406]  # ImageNet RGB means
# )


#### dualbranch model

In [ ]:
def intermediate_layer_size(input, output, n_layers):
    start_exp = (output + 1).bit_length()
    end_exp = (input - 1).bit_length()
    
    total_powers = end_exp - start_exp
    if total_powers < n_layers:
        return None

    result = []
    denominator = n_layers + 1
    half_denominator = denominator // 2

    for i in range(1, n_layers + 1):
        numerator = i * total_powers + half_denominator #works same as rounding
        idx = numerator // denominator
        power = 1 << (start_exp + idx)
        result.append(power)

    return result


intermediate_layer_size(1280+64, 9, 3)

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models

class DualBranchModel(nn.Module):
    def __init__(self, num_outputs=9, dropout_rate=0.3, architecture={'env':'lightweight', 'head':'deep'}):
        super(DualBranchModel, self).__init__()
        self.setup = architecture
        
        self.social_branch = nn.Sequential(
            models.mobilenet_v2(weights=models.MobileNet_V2_Weights.IMAGENET1K_V1).features,
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten()
        )
        soc_feature_dim = 1280
        

        if self.setup['env'] == 'lightweight':
            self.env_branch = nn.Sequential(
                models.mobilenet_v2(weights=models.MobileNet_V2_Weights.IMAGENET1K_V1).features,
                nn.AdaptiveAvgPool2d(1),
                nn.Flatten()
            )
            env_feature_dim = 1280

        elif self.setup['env'] == 'label':
            max_rooms = 30
            self.env_branch = nn.Embedding(max_rooms, 64)
            env_feature_dim = 64


        self.fusion_dim = soc_feature_dim + env_feature_dim

        layers = intermediate_layer_size(soc_feature_dim, num_outputs, 1)
        self.social_classifier = nn.Sequential(
            nn.Linear(soc_feature_dim, layers[0]),
            nn.ReLU(),
            nn.Linear(layers[0], num_outputs)
        )
        layers = intermediate_layer_size(env_feature_dim, num_outputs, 1)
        self.env_classifier = nn.Sequential(
            nn.Linear(env_feature_dim, layers[0]),
            nn.ReLU(),
            nn.Linear(layers[0], num_outputs)
        )
        
        if self.setup['head'] == 'deep':
            self.head = nn.Sequential(
                nn.Linear(self.fusion_dim, 512),
                nn.BatchNorm1d(512),
                nn.ReLU(),
                nn.Dropout(dropout_rate),
                
                nn.Linear(512, 256),
                nn.BatchNorm1d(256),
                nn.ReLU(),
                nn.Dropout(dropout_rate),
                
                nn.Linear(256, 128),
                nn.BatchNorm1d(128),
                nn.ReLU(),
                nn.Dropout(dropout_rate),
                
                nn.Linear(128, num_outputs)
            )
        elif self.setup['head'] == 'shallow':
            self.head = nn.Sequential(
                nn.Linear(self.fusion_dim, 512),
                nn.ReLU(),
                nn.Linear(512, 256),
                nn.ReLU(),
                nn.Linear(256, num_outputs)
            )
        
    def forward(self, social_img, env_img):
        social_features = self.social_branch(social_img)
        env_features = self.env_branch(env_img)
        
        fused_features = torch.cat([social_features, env_features], dim=1)
        scores = self.head(fused_features)

        social_class = self.social_classifier(social_features.detach())
        env_class = self.env_classifier(env_features.detach())
        
        return {
            'output': scores,
            'invariant_domain': social_class,
            'specific_domain': env_class,
            'invariant_feats': social_features,
            'specific_feats': env_features
        }

### K Fold Cross Validation

In [ ]:
def combine_fold_histories(fold_history):
    """
    Combine multiple training histories by averaging them element-wise.
    Handles both simple lists and lists of dictionaries.
    """
    combined_history = {}
    metric_keys = list(fold_history[0].keys())
    
    for key in metric_keys:
        # Check if this metric contains dictionaries
        first_element = fold_history[0][key][0] if fold_history[0][key] else None
        
        if isinstance(first_element, dict):
            # Handle lists of dictionaries (train_epoch_metrics, grad_norms)
            combined_history[key] = average_list_of_dicts(fold_history, key)
        else:
            # Handle simple lists (train_epoch_loss, val_epoch_loss, cross_domain_val)
            stacked_metrics = np.stack([fold_history[fold][key] for fold in fold_history])
            combined_history[key] = np.mean(stacked_metrics, axis=0).tolist()
    
    return combined_history

def average_list_of_dicts(fold_history, metric_key):
    """
    Average a list of dictionaries across folds.
    """
    # Get the dictionary keys from the first fold's first epoch
    dict_keys = list(fold_history[0][metric_key][0].keys())
    
    # Convert each fold's list of dicts to a 2D numpy array
    fold_arrays = []
    for fold in fold_history:
        # Convert list of dicts to 2D array: [epochs, sub_metrics]
        fold_array = np.array([[epoch_dict[k] for k in dict_keys] 
                              for epoch_dict in fold_history[fold][metric_key]])
        fold_arrays.append(fold_array)
    
    # Stack all folds and average: [folds, epochs, sub_metrics] -> [epochs, sub_metrics]
    stacked = np.stack(fold_arrays)
    averaged = np.mean(stacked, axis=0)
    
    # Convert back to list of dictionaries
    result = []
    for epoch_values in averaged:
        epoch_dict = dict(zip(dict_keys, epoch_values))
        result.append(epoch_dict)
    
    return result

In [ ]:
from sklearn.model_selection import StratifiedKFold

df = pd.read_pickle("../data/pepper_data.pkl")
df['image_path'] = '../' + df['image_path']

# Initialize fold column
df['fold'] = -1

# Get unique image paths for splitting
unique_images_df = df[['image_path', 'domain']].reset_index(drop=True)
unique_images_df = unique_images_df[~unique_images_df['image_path'].isin(test_split_idx)]

# Create stratified 5-fold splits based on domain
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)

# Assign fold numbers based on domain stratification
for fold, (_, val_idx) in enumerate(skf.split(unique_images_df['image_path'], unique_images_df['domain'])):
    val_image_paths = unique_images_df.iloc[val_idx]['image_path'].tolist()
    df.loc[df['image_path'].isin(val_image_paths), 'fold'] = fold

# Verify domain distribution across folds
print("Domain distribution across folds:")
print(df.groupby(['fold', 'domain']).size().unstack(fill_value=0))

In [ ]:
fold_loaders = create_crossvalidation_loaders(df, 5, batch_sizes=(32, 64, 64), resize_img_to=(512, 288))

## Training

### setup

In [ ]:
# For LGRBaseline
def baseline_batch(model, batch, device, detach_base, binary, full_replay, loss_params, **kwargs):
    inputs, labels, _ = batch
    inputs, labels = inputs.to(device), labels.to(device)
    outputs = model(inputs)['output']
    loss = kwargs['mse_criterion'](outputs, labels)
    loss.backward()
    metrics = {}
    return loss, metrics

#For DANN
def dann_batch(model, batch, device, detach_base, binary, full_replay, loss_params={'head':0.5, 'social':0.5}, **kwargs):
    inputs, labels, domain_labels = batch
    inputs, labels = inputs.to(device), labels.to(device)
    domain_to_idx = kwargs['domain_to_idx']
    domain_labels = torch.tensor([domain_to_idx[d] for d in domain_labels], device=device)
    mse_criterion = kwargs['mse_criterion']
    bce_criterion = kwargs['bce_criterion']
    alpha = kwargs['alpha']

    current_domain = kwargs['current_domain']
    current_binary_labels = (domain_labels == domain_to_idx[current_domain]).float()
    is_first_domain = bool(domain_to_idx[current_domain] == 0)

    outputs = model(inputs, alpha=alpha, is_first_domain=is_first_domain)

    task_loss = mse_criterion(outputs['output'], labels)

    if is_first_domain:
        inv_domain_loss = 0
        inv_acc = 0
    else:
        inv_domain_loss = bce_criterion(outputs['invariant_domain'].squeeze(), current_binary_labels)
        preds = (outputs['invariant_domain'].squeeze() > 0).float()
        inv_acc = (preds == current_binary_labels).float().mean().item()
    
    total_loss = (loss_params['head'] * task_loss +
                    loss_params['social'] * inv_domain_loss)

    total_loss.backward()
    
    metrics = {
        'task_loss': task_loss.item(),
        'inv_domain': 0 if is_first_domain else inv_domain_loss.item(),
        'inv_acc': inv_acc
    }
    return total_loss, metrics

def heuristic_dualbranch_batch(model, batch, device, detach_base, binary, full_replay, loss_params={}, **kwargs):
    inputs1, inputs2, labels, domain_labels = batch
    inputs1, inputs2, labels, domain_labels = inputs1.to(device), inputs2.to(device), labels.to(device), domain_labels.to(device)
    domain_to_idx = kwargs['domain_to_idx']
    # domain_labels = torch.tensor([domain_to_idx[d] for d in domain_labels], device=device)
    mse_criterion = kwargs['mse_criterion']
    ce_criterion = kwargs['ce_criterion']

    outputs = model(inputs1, inputs2)

    loss = mse_criterion(outputs['output'], labels)
    loss.backward()

    class_optimiser = kwargs['class_optimizer']
    class_optimiser.zero_grad()
    inv_domain_loss = ce_criterion(outputs['invariant_domain'], domain_labels)
    spec_domain_loss = ce_criterion(outputs['specific_domain'], domain_labels)
    (inv_domain_loss + spec_domain_loss).backward()
    class_optimiser.step()
    inv_acc = (outputs['invariant_domain'].argmax(1) == domain_labels).float().mean().item()
    spec_acc = (outputs['specific_domain'].argmax(1) == domain_labels).float().mean().item()
    
    metrics = {
        'inv_domain': inv_domain_loss.item(),
        'spec_domain': spec_domain_loss.item(),
        'inv_acc': inv_acc,
        'spec_acc': spec_acc
    }
    return loss, metrics


# For DualBranchNet
def dualbranch_batch(model, batch, device, detach_base, binary, full_replay, loss_params={'head': 1, 'social': 0.5, 'room': 0.2}, **kwargs):
    inputs, labels, domain_labels = batch
    inputs, labels = inputs.to(device), labels.to(device)
    domain_to_idx = kwargs['domain_to_idx']
    domain_labels = torch.tensor([domain_to_idx[d] for d in domain_labels], device=device)
    mse_criterion = kwargs['mse_criterion']
    ce_criterion = kwargs['ce_criterion']
    cos_criterion = kwargs['cos_criterion']
    alpha = kwargs['alpha']
    if binary:
        bce_criterion = kwargs['bce_criterion']

    # Split batch
    current_domain = kwargs['current_domain']
    current_binary_labels = (domain_labels == domain_to_idx[current_domain]).float()
    current_mask = (domain_labels == domain_to_idx[current_domain])

    if full_replay:
        current_mask = torch.ones_like(current_mask, dtype=torch.bool)

    replay_mask = ~current_mask

    # 1. Current samples: update all parameters
    if current_mask.any():
        inputs_current = inputs[current_mask]
        labels_current = labels[current_mask]
        domain_labels_current = domain_labels[current_mask]

        outputs_current = model(inputs_current, alpha=alpha)
        inv_feats = outputs_current['invariant_feats']
        spec_feats = outputs_current['specific_feats']

        task_loss = mse_criterion(outputs_current['output'], labels_current)
        if binary:
            inv_domain_loss = bce_criterion(outputs_current['invariant_domain'].squeeze(), current_binary_labels[current_mask])
        else:
            inv_domain_loss = ce_criterion(outputs_current['invariant_domain'], domain_labels_current)
        spec_domain_loss = ce_criterion(outputs_current['specific_domain'], domain_labels_current)
        similarity_loss = cos_criterion(inv_feats, spec_feats)
        
        total_loss = (loss_params['head'] * task_loss +
                      loss_params['social'] * inv_domain_loss +
                      loss_params['room'] * spec_domain_loss)

        total_loss.backward(retain_graph= not full_replay)
        
        if binary:
            # Threshold at 0 (sigmoid(0) = 0.5)
            preds = (outputs_current['invariant_domain'].squeeze() > 0).float()
            inv_acc = (preds == current_binary_labels[current_mask]).float().mean().item()
        else:
            inv_acc = (outputs_current['invariant_domain'].argmax(1) == domain_labels_current).float().mean().item()
        spec_acc = (outputs_current['specific_domain'].argmax(1) == domain_labels_current).float().mean().item()
    else:
        total_loss = torch.tensor(0.0, device=device)
        inv_acc = 0.0
        spec_acc = 0.0
        task_loss = torch.tensor(0.0, device=device)
        inv_domain_loss = torch.tensor(0.0, device=device)
        spec_domain_loss = torch.tensor(0.0, device=device)
        similarity_loss = torch.tensor(0.0, device=device)

    # 2. Replay samples: update only specific branch + head
    if replay_mask.any():
        inputs_replay = inputs[replay_mask]
        labels_replay = labels[replay_mask]
        domain_labels_replay = domain_labels[replay_mask]

        #no_grad, unlike requires_grad=False, detaches all elements from the gradient computation graph
        with torch.no_grad():
            base_replay = model.backbone(inputs_replay)
            base_replay = model.pool(base_replay).flatten(1)
            inv_feats_replay = model.invariant(base_replay)

        specific_feats = model.specific(base_replay)
        spec_domain_pred = model.specific_domain_classifier(specific_feats)
        
        if detach_base:
            combined = torch.cat([inv_feats_replay, specific_feats, base_replay], dim=1)
        else:
            combined = torch.cat([inv_feats_replay, specific_feats], dim=1)  
        
        scores = model.head(combined)
        
        task_loss_replay = mse_criterion(scores, labels_replay)
        spec_domain_loss_replay = ce_criterion(spec_domain_pred, domain_labels_replay)
        total_loss_replay = task_loss_replay + 0.2 * spec_domain_loss_replay
        
        total_loss_replay.backward()
    
    metrics = {
        'task_loss': task_loss.item(),
        'inv_domain': inv_domain_loss.item(),
        'spec_domain': spec_domain_loss.item(),
        'similarity': similarity_loss.item(),
        'inv_acc': inv_acc,
        'spec_acc': spec_acc,
        'replay_count': replay_mask.sum().item(),
        'current_count': current_mask.sum().item()
    }
    return total_loss, metrics


In [ ]:
#TODO: change dataset to return torch domain labels indexes not strings
def evaluate_model(model, dataloader, criterion, device , tsne=None):
    model.eval()
    total_loss = 0.0
    total_samples = 0
    with torch.no_grad():
        for batch in dataloader:
            if len(batch) == 4:
                inputs1, inputs2, labels, domain_labels = batch
                inputs1 = inputs1.to(device, dtype=torch.float32)
                inputs2 = inputs2.to(device, dtype=torch.float32)
                labels = labels.to(device, dtype=torch.float32)
                inputs = (inputs1, inputs2)
            elif len(batch) == 3:
                inputs, labels, _ = batch
                inputs = inputs.to(device, dtype=torch.float32)
                labels = labels.to(device, dtype=torch.float32)
                inputs = (inputs,)
            else:
                raise ValueError(f"Batch contains {len(batch)} objects. Should contain 3 or 4 - image/two, labels, domain_labels")

            outputs = model(*inputs)['output']

            loss = criterion(outputs, labels)

            total_loss += loss.item() * inputs[0].size(0)
            total_samples += inputs[0].size(0)
            
            if tsne:
                tsne['social'].append(outputs['invariant_feats'].cpu())
                tsne['environmental'].append(outputs['specific_feats'].cpu())
                tsne['domains'].append(domain_labels.cpu())

            val_loss = total_loss / total_samples

    return (val_loss, tsne) if tsne else val_loss

def cross_domain_validation(model, domain_dataloaders, criterion, device, tsne=None):
    results = {}
    for domain, loaders in domain_dataloaders.items():
        val_loader = loaders['val']
        if tsne:
            val_loss, tsne = evaluate_model(model, val_loader, criterion, device, tsne)
        else:
            val_loss = evaluate_model(model, val_loader, criterion, device)
        results[domain] = val_loss
    return (results, tsne) if tsne else results

def average_metrics(metrics_list):
    # metrics_list: list of dicts, each dict contains metrics for a batch
    if not metrics_list:
        return {}
    keys = metrics_list[0].keys()
    avg_metrics = {}
    for k in keys:
        avg_metrics[k] = float(np.mean([m[k] for m in metrics_list if k in m]))
    return avg_metrics

def collect_tsne_features(model, loader, device):
    model.eval()
    all_inv, all_spec, all_domains = [], [], []
    with torch.no_grad():
        for domain, loaders in domain_dataloaders.items():
            loader = loaders['val']
            for x, _, d in loader:
                x = x.to(device)
                out = model(x)
                all_inv.append(out['invariant_feats'].cpu())
                all_domains += list(d)
    inv_feats = torch.cat(all_inv, dim=0).numpy()
    return inv_feats, all_domains


def collect_gradients(model):
    grad_norms = {}
    for name, param in model.named_parameters():
        if param.grad is not None and not name.startswith("backbone"):
            module = name.split('.')[0]
            norm = param.grad.norm(2).item()
            if module not in grad_norms:
                grad_norms[module] = []
            grad_norms[module].append(norm)
    # Take mean per module
    grad_norms = {k: float(np.mean(v)) for k, v in grad_norms.items()}
    return grad_norms


In [ ]:
import pickle
import torch
from tqdm.notebook import tqdm, trange

def unified_train_loop(
    model, domains, domain_dataloaders, buffer, optimizer, writer, device,
    batch_fn, batch_kwargs, loss_params, num_epochs=5, exp_name="exp", gradient_clipping=False, detach_base=False, binary=False, full_replay=False, collect_tsne_data=False, restart={}, eval_buffer=False
):
    start_domain_idx = 0
    global_step = 0
    history = {
        'train_epoch_loss': [],
        'val_epoch_loss': [],
        'val_buffer_epoch_loss': [],
        'train_epoch_metrics': [],
        'cross_domain_val': [],
        'grad_norms': [],
    }
    
    if restart:
        # Populate history
        global_step = restart['global_step']
        history = restart['history']
        # Populate buffer
        start_domain_idx = np.where(domains == restart['domain'])[0][0]
        for domain_idx, current_domain in enumerate(domains[:start_domain_idx]):
            buffer.update_buffer(current_domain, domain_dataloaders[current_domain]['train'].dataset) 
        print(f"Restarting from domain {restart['domain']} index {start_domain_idx}")
        print(f"Buffer: {buffer.get_domain_distribution()}")         
        

    for domain_idx, current_domain in enumerate(tqdm(domains[start_domain_idx:], desc=f"Total training"), start=start_domain_idx):
        train_loader = buffer.get_loader_with_replay(current_domain, domain_dataloaders[current_domain]['train'])
        if eval_buffer:
            eval_loader = eval_buffer.get_loader_with_replay(current_domain, domain_dataloaders[current_domain]['val'])
        else:
            eval_loader = domain_dataloaders[current_domain]['val']
        len_dataloader = len(train_loader)
        
        for epoch in trange(num_epochs, desc=f"Current domain {current_domain}"):
            model.train()
            epoch_loss = 0.0
            samples = 0
            batch_metrics_list = []
            
            # for batch_idx, batch in enumerate(train_loader):
            for batch_idx, batch in enumerate(tqdm(train_loader, desc=f"Current epoch {epoch}", leave=False)):
                if not batch_kwargs['alpha']:
                    p = (epoch * len_dataloader + batch_idx) / (num_epochs * len_dataloader)
                    alpha = 2. / (1. + np.exp(-10 * p)) - 1
                else:
                    alpha = batch_kwargs['alpha']

                optimizer.zero_grad()
                loss, metrics = batch_fn(model, batch, device, detach_base, binary, full_replay, loss_params, **{**batch_kwargs, 'current_domain': current_domain, 'alpha':alpha})
                if gradient_clipping:
                    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                optimizer.step()
                batch_size = batch[0].size(0)
                epoch_loss += loss.item() * batch_size
                samples += batch_size
                global_step += 1
                batch_metrics_list.append(metrics)
                # # TensorBoard logging (every 10 batches)
                # if writer and batch_idx % 10 == 0:
                #     writer.add_scalar(f'{exp_name}/train_loss', loss.item(), global_step)
                #     for k, v in metrics.items():
                #         writer.add_scalar(f'{exp_name}/train_{k}', v, global_step)
            avg_epoch_loss = epoch_loss / samples
            # writer.add_scalar(f'{exp_name}/train_epoch_loss', avg_epoch_loss, global_step)
            history['train_epoch_loss'].append(avg_epoch_loss)
            # Average batch metrics for this epoch
            avg_metrics = average_metrics(batch_metrics_list)
            history['train_epoch_metrics'].append(avg_metrics)

            # Collect gradients
            grad_norms = collect_gradients(model)
            history['grad_norms'].append(grad_norms)

            # Validation on current domain
            val_loss = evaluate_model(model, domain_dataloaders[current_domain]['val'], batch_kwargs['mse_criterion'], device)
            val_loss_buffer = evaluate_model(model, eval_loader, batch_kwargs['mse_criterion'], device)
            # writer.add_scalar(f'{exp_name}/val_epoch_loss', val_loss, global_step)
            history['val_epoch_loss'].append(val_loss)
            history['val_buffer_epoch_loss'].append(val_loss_buffer)

            # Collect data for t-SNE domain separation graphs
            if collect_tsne_data:
                inv_feats, domain_labels = collect_tsne_features(model, domain_dataloaders, device)
                tsne_data = {
                    'inv_feats': inv_feats,
                    'domain_labels': domain_labels
                }
            else:
                tsne_data = None

            # Cross-domain validation (after each domain)
            if epoch == num_epochs-1:
                if collect_tsne_data:
                    tsne = {'social': [], 'env': [], 'domains': []}
                    cross_val, tsne_data = cross_domain_validation(model, domain_dataloaders, batch_kwargs['mse_criterion'], device, tsne)
                else:
                    cross_val = cross_domain_validation(model, domain_dataloaders, batch_kwargs['mse_criterion'], device)
                history['cross_domain_val'].append(cross_val)

                # Only save last model per domain to save space
                torch.save({
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'history': history,
                    'tsne' : tsne_data,
                }, f"../checkpoints/{exp_name}_domain{current_domain}_epoch{epoch}_step{global_step}.pt")
            # else:
            #     # Save metrics
            #     torch.save({
            #         # 'model_state_dict': model.state_dict(),
            #         # 'optimizer_state_dict': optimizer.state_dict(),
            #         'history': history,
            #         'tsne' : tsne_data,
            #     }, f"../checkpoints/{exp_name}_domain{current_domain}_epoch{epoch}_step{global_step}.pt")
            with open(f"../checkpoints/{exp_name}_history.pkl", "wb") as f:
                pickle.dump(history, f)
            
        buffer.update_buffer(current_domain, domain_dataloaders[current_domain]['train'].dataset)
        eval_buffer.update_buffer(current_domain, domain_dataloaders[current_domain]['val'].dataset)
    return history


### runs

In [ ]:
import datetime
from torch.utils.tensorboard import SummaryWriter

# Setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
domains = df['domain'].unique()
domain_to_idx = {d: i for i, d in enumerate(domains)}

# # For baseline model
# baseline_model = LGRBaseline().to(device)
# optimizer = torch.optim.Adam(baseline_model.parameters(), lr=1e-3)
# buffer = NaiveRehearsalBuffer(buffer_size=1000)
# exp_name = f"baselinemodel_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}"
# writer = SummaryWriter(log_dir=f"../tensorboard/{exp_name}")

# baseline_kwargs = {'mse_criterion': nn.MSELoss()}
# unified_train_loop(
#     model=baseline_model,
#     domains=domains,
#     domain_dataloaders=domain_dataloaders,
#     buffer=buffer,
#     optimizer=optimizer,
#     writer=writer,
#     device=device,
#     batch_fn=baseline_batch,
#     batch_kwargs=baseline_kwargs,
#     num_epochs=10,
#     exp_name=exp_name
# )

# For DualBranchNet
dual_model = DualBranchNet(num_domains=len(domains)).to(device)
optimizer = torch.optim.Adam(dual_model.parameters(), lr=1e-3)
buffer = NaiveRehearsalBuffer(buffer_size=1000)

def cos_criterion(a, b):
    return (1 - torch.abs(nn.CosineSimilarity()(a, b))).mean()
    # return (nn.CosineSimilarity()(a, b) ** 2).mean()

exp_name = f"nores_dualbranchmodel_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}"
writer = SummaryWriter(log_dir=f"../tensorboard/{exp_name}")

dualbranch_kwargs = {
    'mse_criterion': nn.MSELoss(),
    'ce_criterion': nn.CrossEntropyLoss(),
    'cos_criterion': cos_criterion,
    'domain_to_idx': domain_to_idx
}
unified_train_loop(
    model=dual_model,
    domains=domains,
    domain_dataloaders=domain_dataloaders,
    buffer=buffer,
    optimizer=optimizer,
    writer=writer,
    device=device,
    batch_fn=dualbranch_batch,
    batch_kwargs=dualbranch_kwargs,
    num_epochs=10,
    exp_name=exp_name,
    gradient_clipping=False
)


# For DualBranchNet
dual_model = DualBranchNet(num_domains=len(domains)).to(device)
optimizer = torch.optim.Adam(dual_model.parameters(), lr=1e-3)
buffer = NaiveRehearsalBuffer(buffer_size=1000)

def cos_criterion(a, b):
    return (1 - torch.abs(nn.CosineSimilarity()(a, b))).mean()
    # return (nn.CosineSimilarity()(a, b) ** 2).mean()

exp_name = f"nores_gradclip_dualbranchmodel_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}"
writer = SummaryWriter(log_dir=f"../tensorboard/{exp_name}")

dualbranch_kwargs = {
    'mse_criterion': nn.MSELoss(),
    'ce_criterion': nn.CrossEntropyLoss(),
    'cos_criterion': cos_criterion,
    'domain_to_idx': domain_to_idx
}
unified_train_loop(
    model=dual_model,
    domains=domains,
    domain_dataloaders=domain_dataloaders,
    buffer=buffer,
    optimizer=optimizer,
    writer=writer,
    device=device,
    batch_fn=dualbranch_batch,
    batch_kwargs=dualbranch_kwargs,
    num_epochs=10,
    exp_name=exp_name,
    gradient_clipping=True
)

# For DualBranchNet
dual_model = DualBranchNet(weights_init=True, weights_norm=True).to(device)
optimizer = torch.optim.Adam(dual_model.parameters(), lr=1e-3)
buffer = NaiveRehearsalBuffer(buffer_size=1000)

def cos_criterion(a, b):
    return (1 - torch.abs(nn.CosineSimilarity()(a, b))).mean()
    # return (nn.CosineSimilarity()(a, b) ** 2).mean()

exp_name = f"nores_winit_wnorm_dualbranchmodel_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}"
writer = SummaryWriter(log_dir=f"../tensorboard/{exp_name}")

dualbranch_kwargs = {
    'mse_criterion': nn.MSELoss(),
    'ce_criterion': nn.CrossEntropyLoss(),
    'cos_criterion': cos_criterion,
    'domain_to_idx': domain_to_idx
}
unified_train_loop(
    model=dual_model,
    domains=domains,
    domain_dataloaders=domain_dataloaders,
    buffer=buffer,
    optimizer=optimizer,
    writer=writer,
    device=device,
    batch_fn=dualbranch_batch,
    batch_kwargs=dualbranch_kwargs,
    num_epochs=10,
    exp_name=exp_name,
    gradient_clipping=False
)

# For DualBranchNet
dual_model = DualBranchNet(weights_init=True, weights_norm=True).to(device)
optimizer = torch.optim.Adam(dual_model.parameters(), lr=1e-3)
buffer = NaiveRehearsalBuffer(buffer_size=1000)

def cos_criterion(a, b):
    return (1 - torch.abs(nn.CosineSimilarity()(a, b))).mean()
    # return (nn.CosineSimilarity()(a, b) ** 2).mean()

exp_name = f"nores_gradclip_winit_wnorm_dualbranchmodel_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}"
writer = SummaryWriter(log_dir=f"../tensorboard/{exp_name}")

dualbranch_kwargs = {
    'mse_criterion': nn.MSELoss(),
    'ce_criterion': nn.CrossEntropyLoss(),
    'cos_criterion': cos_criterion,
    'domain_to_idx': domain_to_idx
}
unified_train_loop(
    model=dual_model,
    domains=domains,
    domain_dataloaders=domain_dataloaders,
    buffer=buffer,
    optimizer=optimizer,
    writer=writer,
    device=device,
    batch_fn=dualbranch_batch,
    batch_kwargs=dualbranch_kwargs,
    num_epochs=10,
    exp_name=exp_name,
    gradient_clipping = True
)


In [ ]:
import datetime
from torch.utils.tensorboard import SummaryWriter

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
domains = df['domain'].unique()
domain_to_idx = {d: i for i, d in enumerate(domains)}

dual_model = DualBranchNet_deep(weights_init=True, weights_norm=False).to(device)
optimizer = torch.optim.Adam(dual_model.parameters(), lr=1e-3)
buffer = NaiveRehearsalBuffer(buffer_size=1000)

def cos_criterion(a, b):
    return (1 - torch.abs(nn.CosineSimilarity()(a, b))).mean()
    # return (nn.CosineSimilarity()(a, b) ** 2).mean()

exp_name = f"deep_dualbranchmodel_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}"
writer = SummaryWriter(log_dir=f"../tensorboard/{exp_name}")

dualbranch_kwargs = {
    'mse_criterion': nn.MSELoss(),
    'ce_criterion': nn.CrossEntropyLoss(),
    'cos_criterion': cos_criterion,
    'domain_to_idx': domain_to_idx
}
unified_train_loop(
    model=dual_model,
    domains=domains,
    domain_dataloaders=domain_dataloaders,
    buffer=buffer,
    optimizer=optimizer,
    writer=writer,
    device=device,
    batch_fn=dualbranch_batch,
    batch_kwargs=dualbranch_kwargs,
    num_epochs=10,
    exp_name=exp_name,
    gradient_clipping=True
)


dual_model = DualBranchNet_deep(weights_init=True, weights_norm=True).to(device)
optimizer = torch.optim.Adam(dual_model.parameters(), lr=1e-3)
buffer = NaiveRehearsalBuffer(buffer_size=1000)

def cos_criterion(a, b):
    return (1 - torch.abs(nn.CosineSimilarity()(a, b))).mean()
    # return (nn.CosineSimilarity()(a, b) ** 2).mean()

exp_name = f"deep_norm_dualbranchmodel_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}"
writer = SummaryWriter(log_dir=f"../tensorboard/{exp_name}")

dualbranch_kwargs = {
    'mse_criterion': nn.MSELoss(),
    'ce_criterion': nn.CrossEntropyLoss(),
    'cos_criterion': cos_criterion,
    'domain_to_idx': domain_to_idx
}
unified_train_loop(
    model=dual_model,
    domains=domains,
    domain_dataloaders=domain_dataloaders,
    buffer=buffer,
    optimizer=optimizer,
    writer=writer,
    device=device,
    batch_fn=dualbranch_batch,
    batch_kwargs=dualbranch_kwargs,
    num_epochs=10,
    exp_name=exp_name,
    gradient_clipping=True
)

In [ ]:
import datetime
from torch.utils.tensorboard import SummaryWriter

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
domains = df['domain'].unique()
domain_to_idx = {d: i for i, d in enumerate(domains)}

dual_model = DualBranchNet_deep(weights_init=False, weights_norm=False, layer_norm=False, detach_base=True).to(device)
optimizer = torch.optim.Adam(dual_model.parameters(), lr=1e-3)
buffer = NaiveRehearsalBuffer(buffer_size=1000)

def cos_criterion(a, b):
    return (1 - torch.abs(nn.CosineSimilarity()(a, b))).mean()
    # return (nn.CosineSimilarity()(a, b) ** 2).mean()

exp_name = f"bbdetach_deep_dualbranchmodel_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}"
writer = SummaryWriter(log_dir=f"../tensorboard/{exp_name}")

dualbranch_kwargs = {
    'mse_criterion': nn.MSELoss(),
    'ce_criterion': nn.CrossEntropyLoss(),
    'cos_criterion': cos_criterion,
    'domain_to_idx': domain_to_idx
}
unified_train_loop(
    model=dual_model,
    domains=domains,
    domain_dataloaders=domain_dataloaders,
    buffer=buffer,
    optimizer=optimizer,
    writer=writer,
    device=device,
    batch_fn=dualbranch_batch,
    batch_kwargs=dualbranch_kwargs,
    num_epochs=10,
    exp_name=exp_name,
    gradient_clipping=True
)

In [ ]:
import datetime
from torch.utils.tensorboard import SummaryWriter

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
domains = df['domain'].unique()
domain_to_idx = {d: i for i, d in enumerate(domains)}

dual_model = DualBranchNet_deep(weights_init=True, weights_norm=False, layer_norm=False, detach_base=True).to(device)
optimizer = torch.optim.Adam(dual_model.parameters(), lr=1e-3)
buffer = NaiveRehearsalBuffer(buffer_size=1000)

def cos_criterion(a, b):
    return (1 - torch.abs(nn.CosineSimilarity()(a, b))).mean()
    # return (nn.CosineSimilarity()(a, b) ** 2).mean()

exp_name = f"bdetach_batch16_dualbranchmodel_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}"
writer = SummaryWriter(log_dir=f"../tensorboard/{exp_name}")

dualbranch_kwargs = {
    'mse_criterion': nn.MSELoss(),
    'ce_criterion': nn.CrossEntropyLoss(),
    'cos_criterion': cos_criterion,
    'domain_to_idx': domain_to_idx
}

domain_dataloaders = {}
for domain in domains:
    domain_df = df[df['domain'] == domain]
    loaders = create_dataloaders(domain_df, batch_sizes=(16, 64, 64), resize_img_to=(128, 128), seed=SEED)  #TODO should be (384, 216) to retain scale or (224, 224) for best performance on MobileNet
    domain_dataloaders[domain] = loaders

unified_train_loop(
    model=dual_model,
    domains=domains,
    domain_dataloaders=domain_dataloaders,
    buffer=buffer,
    optimizer=optimizer,
    writer=writer,
    device=device,
    batch_fn=dualbranch_batch,
    batch_kwargs=dualbranch_kwargs,
    num_epochs=10,
    exp_name=exp_name,
    gradient_clipping=True,
    detach_base=True
)

domain_dataloaders = {}
for domain in domains:
    domain_df = df[df['domain'] == domain]
    loaders = create_dataloaders(domain_df, batch_sizes=(32, 64, 64), resize_img_to=(128, 128), seed=SEED)  #TODO should be (384, 216) to retain scale or (224, 224) for best performance on MobileNet
    domain_dataloaders[domain] = loaders


dual_model = DualBranchNet_deep(weights_init=False, weights_norm=False, layer_norm=False, detach_base=True, freeze_base='partial').to(device)
optimizer = torch.optim.Adam(dual_model.parameters(), lr=1e-3)
buffer = NaiveRehearsalBuffer(buffer_size=1000)

def cos_criterion(a, b):
    return (1 - torch.abs(nn.CosineSimilarity()(a, b))).mean()
    # return (nn.CosineSimilarity()(a, b) ** 2).mean()

exp_name = f"bdetach_pfrozen_dualbranchmodel_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}"
writer = SummaryWriter(log_dir=f"../tensorboard/{exp_name}")

dualbranch_kwargs = {
    'mse_criterion': nn.MSELoss(),
    'ce_criterion': nn.CrossEntropyLoss(),
    'cos_criterion': cos_criterion,
    'domain_to_idx': domain_to_idx
}
unified_train_loop(
    model=dual_model,
    domains=domains,
    domain_dataloaders=domain_dataloaders,
    buffer=buffer,
    optimizer=optimizer,
    writer=writer,
    device=device,
    batch_fn=dualbranch_batch,
    batch_kwargs=dualbranch_kwargs,
    num_epochs=10,
    exp_name=exp_name,
    gradient_clipping=True,
    detach_base=True
)

dual_model = DualBranchNet_deep(weights_init=False, weights_norm=False, layer_norm=False, detach_base=False, freeze_base='full').to(device)
optimizer = torch.optim.Adam(dual_model.parameters(), lr=1e-3)
buffer = NaiveRehearsalBuffer(buffer_size=1000)

def cos_criterion(a, b):
    return (1 - torch.abs(nn.CosineSimilarity()(a, b))).mean()
    # return (nn.CosineSimilarity()(a, b) ** 2).mean()

exp_name = f"ffrozen_dualbranchmodel_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}"
writer = SummaryWriter(log_dir=f"../tensorboard/{exp_name}")

dualbranch_kwargs = {
    'mse_criterion': nn.MSELoss(),
    'ce_criterion': nn.CrossEntropyLoss(),
    'cos_criterion': cos_criterion,
    'domain_to_idx': domain_to_idx
}
unified_train_loop(
    model=dual_model,
    domains=domains,
    domain_dataloaders=domain_dataloaders,
    buffer=buffer,
    optimizer=optimizer,
    writer=writer,
    device=device,
    batch_fn=dualbranch_batch,
    batch_kwargs=dualbranch_kwargs,
    num_epochs=10,
    exp_name=exp_name,
    gradient_clipping=True,
    detach_base=False
)

In [ ]:
import datetime
from torch.utils.tensorboard import SummaryWriter

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
domains = df['domain'].unique()
domain_to_idx = {d: i for i, d in enumerate(domains)}

dual_model = DualBranchNet_binary(weights_init=False, weights_norm=False, layer_norm=False, detach_base=False, freeze_base='full', explicit_grl=False).to(device)
optimizer = torch.optim.Adam(dual_model.parameters(), lr=1e-3)
buffer = NaiveRehearsalBuffer(buffer_size=1000)

def cos_criterion(a, b):
    return (1 - torch.abs(nn.CosineSimilarity()(a, b))).mean()
    # return (nn.CosineSimilarity()(a, b) ** 2).mean()

exp_name = f"ffrozen_binary_dualbranchmodel_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}"
writer = SummaryWriter(log_dir=f"../tensorboard/{exp_name}")

dualbranch_kwargs = {
    'mse_criterion': nn.MSELoss(),
    'ce_criterion': nn.CrossEntropyLoss(),
    'cos_criterion': cos_criterion,
    'domain_to_idx': domain_to_idx,
    'bce_criterion': nn.BCEWithLogitsLoss()
}
unified_train_loop(
    model=dual_model,
    domains=domains,
    domain_dataloaders=domain_dataloaders,
    buffer=buffer,
    optimizer=optimizer,
    writer=writer,
    device=device,
    batch_fn=dualbranch_batch,
    batch_kwargs=dualbranch_kwargs,
    num_epochs=10,
    exp_name=exp_name,
    gradient_clipping=True,
    detach_base=False,
    binary = True,
    full_replay = False
)

dual_model = DualBranchNet_binary(weights_init=False, weights_norm=False, layer_norm=False, detach_base=False, freeze_base='full', explicit_grl=False).to(device)
optimizer = torch.optim.Adam(dual_model.parameters(), lr=1e-3)
buffer = NaiveRehearsalBuffer(buffer_size=1000)

def cos_criterion(a, b):
    return (1 - torch.abs(nn.CosineSimilarity()(a, b))).mean()
    # return (nn.CosineSimilarity()(a, b) ** 2).mean()

exp_name = f"ffrozen_binary_explicitgrl_dualbranchmodel_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}"
writer = SummaryWriter(log_dir=f"../tensorboard/{exp_name}")

dualbranch_kwargs = {
    'mse_criterion': nn.MSELoss(),
    'ce_criterion': nn.CrossEntropyLoss(),
    'cos_criterion': cos_criterion,
    'domain_to_idx': domain_to_idx,
    'bce_criterion': nn.BCEWithLogitsLoss()
}
unified_train_loop(
    model=dual_model,
    domains=domains,
    domain_dataloaders=domain_dataloaders,
    buffer=buffer,
    optimizer=optimizer,
    writer=writer,
    device=device,
    batch_fn=dualbranch_batch,
    batch_kwargs=dualbranch_kwargs,
    num_epochs=10,
    exp_name=exp_name,
    gradient_clipping=True,
    detach_base=False,
    binary=True,
    full_replay = False
)


dual_model = DualBranchNet_binary(weights_init=True, weights_norm=False, layer_norm=False, detach_base=False, freeze_base='full', explicit_grl=True).to(device)
optimizer = torch.optim.Adam(dual_model.parameters(), lr=1e-3)
buffer = NaiveRehearsalBuffer(buffer_size=1000)

def cos_criterion(a, b):
    return (1 - torch.abs(nn.CosineSimilarity()(a, b))).mean()
    # return (nn.CosineSimilarity()(a, b) ** 2).mean()

exp_name = f"freplay_ffrozen_binary_explicitgrl_dualbranchmodel_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}"
writer = SummaryWriter(log_dir=f"../tensorboard/{exp_name}")

dualbranch_kwargs = {
    'mse_criterion': nn.MSELoss(),
    'ce_criterion': nn.CrossEntropyLoss(),
    'cos_criterion': cos_criterion,
    'domain_to_idx': domain_to_idx,
    'bce_criterion': nn.BCEWithLogitsLoss()
}
unified_train_loop(
    model=dual_model,
    domains=domains,
    domain_dataloaders=domain_dataloaders,
    buffer=buffer,
    optimizer=optimizer,
    writer=writer,
    device=device,
    batch_fn=dualbranch_batch,
    batch_kwargs=dualbranch_kwargs,
    num_epochs=10,
    exp_name=exp_name,
    gradient_clipping=True,
    detach_base=False,
    binary=True,
    full_replay = True
)

In [ ]:
from itertools import product
import datetime
from torch.utils.tensorboard import SummaryWriter

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
domains = df['domain'].unique()
domain_to_idx = {d: i for i, d in enumerate(domains)}

testing_scenarios = {
    # 'pretrained_backbone_cnn_branch': ('pretrained', 'simple', 'simple'), 
    # 'linear_branch': ('3conv', 'linear', 'simple'),
    # 'cnn_branch': ('3conv', 'simple', 'simple'),
    # 'adversarial': ('2conv', 'adversarial', 'adversarial'),
    'cnn_specialised_branches': ('3conv', 'special', 'simple')
}

for name, (backbone, branch, end) in testing_scenarios.items():
    print(f"\nTesting: {backbone} + {branch} + {end}")
    dual_model = DualBranchCNNNet(backbone_type=backbone, branch_type=branch, end_type=end, batch_norm=True).to(device)
    optimizer = torch.optim.Adam(dual_model.parameters(), lr=1e-3)
    buffer = NaiveRehearsalBuffer(buffer_size=1000)

    def cos_criterion(a, b):
        return (1 - torch.abs(nn.CosineSimilarity()(a, b))).mean()

    exp_name = f"CNN_{name}_dualbranchmodel_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}"
    writer = SummaryWriter(log_dir=f"../tensorboard/{exp_name}")

    dualbranch_kwargs = {
        'mse_criterion': nn.MSELoss(),
        'ce_criterion': nn.CrossEntropyLoss(),
        'cos_criterion': cos_criterion,
        'domain_to_idx': domain_to_idx,
        'bce_criterion': nn.BCEWithLogitsLoss()
    }
    unified_train_loop(
        model=dual_model,
        domains=domains,
        domain_dataloaders=domain_dataloaders,
        buffer=buffer,
        optimizer=optimizer,
        writer=writer,
        device=device,
        batch_fn=dualbranch_batch,
        batch_kwargs=dualbranch_kwargs,
        num_epochs=10,
        exp_name=exp_name,
        gradient_clipping=False,
        detach_base=False,
        binary = True,
        full_replay = True,
        collect_tsne_data=False
    )


In [ ]:
import datetime
from torch.utils.tensorboard import SummaryWriter

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
domains = df['domain'].unique()
domain_to_idx = {d: i for i, d in enumerate(domains)}

testing_scenarios = {
    'pretrained_simple': ('pretrained', 'simple', '3linear', False), 
    'pretrained_special': ('pretrained', 'special', '3linear', False),
    '3conv_simple': ('3conv', 'simple', '3linear', True),
    '3conv_special': ('3conv', 'special', '3linear', True),
}

for name, (backbone, branch, end, detach_base) in testing_scenarios.items():
    print(f"\nTesting: {backbone} + {branch} + {end}")
    dual_model = DualBranchCNNNet(backbone_type=backbone, branch_type=branch, end_type=end, batch_norm=True).to(device)
    optimizer = torch.optim.Adam(dual_model.parameters(), lr=1e-3)
    buffer = NaiveRehearsalBuffer(buffer_size=1000)

    def cos_criterion(a, b):
        return (1 - torch.abs(nn.CosineSimilarity()(a, b))).mean()

    exp_name = f"CNN_{name}_dualbranchmodel_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}"
    writer = SummaryWriter(log_dir=f"../tensorboard/{exp_name}")

    dualbranch_kwargs = {
        'mse_criterion': nn.MSELoss(),
        'ce_criterion': nn.CrossEntropyLoss(),
        'cos_criterion': cos_criterion,
        'domain_to_idx': domain_to_idx,
        'bce_criterion': nn.BCEWithLogitsLoss()
    }
    unified_train_loop(
        model=dual_model,
        domains=domains,
        domain_dataloaders=domain_dataloaders,
        buffer=buffer,
        optimizer=optimizer,
        writer=writer,
        device=device,
        batch_fn=dualbranch_batch,
        batch_kwargs=dualbranch_kwargs,
        num_epochs=10,
        exp_name=exp_name,
        gradient_clipping=True,
        detach_base=detach_base,
        binary = True,
        full_replay = True,
        collect_tsne_data=False
    )

In [ ]:
import datetime
from torch.utils.tensorboard import SummaryWriter

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
domains = df['domain'].unique()
domain_to_idx = {d: i for i, d in enumerate(domains)}

testing_scenarios = {
    'pretrained_simple_0.25branches': ('pretrained', 'simple', '3linear'), 
    # '3conv_adversarial': ('3conv', 'adversarial', 'adversarial'),
}

for name, (backbone, branch, end) in testing_scenarios.items():
    print(f"\nTesting: {backbone} + {branch} + {end}")
    dual_model = DualBranchCNNNet(backbone_type=backbone, branch_type=branch, end_type=end, batch_norm=True).to(device)
    optimizer = torch.optim.Adam(dual_model.parameters(), lr=1e-3)
    buffer = NaiveRehearsalBuffer(buffer_size=1000)

    def cos_criterion(a, b):
        return (1 - torch.abs(nn.CosineSimilarity()(a, b))).mean()

    exp_name = f"CNN_{name}_dualbranchmodel_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}"
    writer = SummaryWriter(log_dir=f"../tensorboard/{exp_name}")

    dualbranch_kwargs = {
        'mse_criterion': nn.MSELoss(),
        'ce_criterion': nn.CrossEntropyLoss(),
        'cos_criterion': cos_criterion,
        'domain_to_idx': domain_to_idx,
        'bce_criterion': nn.BCEWithLogitsLoss()
    }
    unified_train_loop(
        model=dual_model,
        domains=domains,
        domain_dataloaders=domain_dataloaders,
        buffer=buffer,
        optimizer=optimizer,
        writer=writer,
        device=device,
        batch_fn=dualbranch_batch,
        batch_kwargs=dualbranch_kwargs,
        num_epochs=10,
        exp_name=exp_name,
        gradient_clipping=True,
        detach_base=False,
        binary = True,
        full_replay = True,
        collect_tsne_data=False,
        loss_params = {'head': 0.5, 'social': 0.25, 'room': 0.25}
    )

In [ ]:
import datetime
from torch.utils.tensorboard import SummaryWriter
from tqdm.notebook import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
domains = df['domain'].unique()
domain_to_idx = {d: i for i, d in enumerate(domains)}

testing_scenarios = {
    # 'pretrained_simple': ('pretrained', 'simple', '3linear'), 
    '3conv_adversarial': ('3conv', 'adversarial', 'adversarial'),
}

for name, (backbone, branch, end) in testing_scenarios.items():
    print(f"\nTesting: {backbone} + {branch} + {end}")
    fold_history = {}
    for fold_num, domain_dataloaders in tqdm(fold_loaders.items()):
        dual_model = DualBranchCNNNet(backbone_type=backbone, branch_type=branch, end_type=end, batch_norm=True).to(device)
        optimizer = torch.optim.Adam(dual_model.parameters(), lr=1e-3)
        buffer = NaiveRehearsalBuffer(buffer_size=1000)

        def cos_criterion(a, b):
            return (1 - torch.abs(nn.CosineSimilarity()(a, b))).mean()

        exp_name = f"fold{fold_num}_CNN_{name}_dualbranchmodel_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}"
        writer = SummaryWriter(log_dir=f"../tensorboard/{exp_name}")

        dualbranch_kwargs = {
            'mse_criterion': nn.MSELoss(),
            'ce_criterion': nn.CrossEntropyLoss(),
            'cos_criterion': cos_criterion,
            'domain_to_idx': domain_to_idx,
            'bce_criterion': nn.BCEWithLogitsLoss()
        }
        history = unified_train_loop(
            model=dual_model,
            domains=domains,
            domain_dataloaders=domain_dataloaders,
            buffer=buffer,
            optimizer=optimizer,
            writer=writer,
            device=device,
            batch_fn=dualbranch_batch,
            batch_kwargs=dualbranch_kwargs,
            num_epochs=10,
            exp_name=exp_name,
            gradient_clipping=True,
            detach_base=False,
            binary = True,
            full_replay = True,
            collect_tsne_data=False,
            loss_params = {'head': 0.5, 'social': 0.25, 'room': 0.25}
        )
        fold_history[fold_num] = history
    with open(f"../checkpoints/5foldcrossval_{exp_name}_history.pkl", "wb") as f:
        pickle.dump(fold_history, f)

In [ ]:
import datetime
from torch.utils.tensorboard import SummaryWriter
from tqdm.notebook import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
domains = df['domain'].unique()
domain_to_idx = {d: i for i, d in enumerate(domains)}


dual_model = DualBranchNet_DANN().to(device)
optimizer = torch.optim.Adam(dual_model.parameters(), lr=1e-3)
buffer = NaiveRehearsalBuffer(buffer_size=1000)

def cos_criterion(a, b):
    return (1 - torch.abs(nn.CosineSimilarity()(a, b))).mean()

exp_name = f"DANN_dualbranchmodel_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}"
writer = SummaryWriter(log_dir=f"../tensorboard/{exp_name}")

dualbranch_kwargs = {
    'mse_criterion': nn.MSELoss(),
    'domain_to_idx': domain_to_idx,
    'bce_criterion': nn.BCEWithLogitsLoss()
}
history = unified_train_loop(
    model=dual_model,
    domains=domains,
    domain_dataloaders=domain_dataloaders,
    buffer=buffer,
    optimizer=optimizer,
    writer=writer,
    device=device,
    batch_fn=dann_batch,
    batch_kwargs=dualbranch_kwargs,
    num_epochs=10,
    exp_name=exp_name,
    gradient_clipping=True,
    detach_base=False,
    binary = True,
    full_replay = True,
    collect_tsne_data=False,
    loss_params = {'head': 1, 'social': 2}
)

In [ ]:
import datetime
from torch.utils.tensorboard import SummaryWriter

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
domains = df['domain'].unique()
domain_to_idx = {d: i for i, d in enumerate(domains)}

domain_dataloaders = {}
for domain in domains:
    domain_df = df[df['domain'] == domain]
    loaders, split_idx = create_dataloaders(domain_df, batch_sizes=(32, 64, 64), resize_img_to=(512,288), seed=SEED, return_splits=True)  #512, 288 # 352,128 #LGR had 128,128 MobileNetv2 had 224, 224
    domain_dataloaders[domain] = loaders

baseline_model = LGRBaseline().to(device)
optimizer = torch.optim.Adam(baseline_model.parameters(), lr=1e-3)
buffer = NaiveRehearsalBuffer(buffer_size=1000)

exp_name = f"baselinemodel_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}"
writer = SummaryWriter(log_dir=f"../tensorboard/{exp_name}")

baseline_kwargs = {'mse_criterion': nn.MSELoss()}
unified_train_loop(
    model=baseline_model,
    domains=domains,
    domain_dataloaders=domain_dataloaders,
    buffer=buffer,
    optimizer=optimizer,
    writer=writer,
    device=device,
    batch_fn=baseline_batch,
    batch_kwargs=baseline_kwargs,
    num_epochs=10,
    exp_name=exp_name,
    loss_params={}
)

domain_dataloaders = {}
for domain in domains:
    domain_df = df[df['domain'] == domain]
    loaders, split_idx = create_dataloaders(domain_df, batch_sizes=(32, 64, 64), resize_img_to=(256,144), seed=SEED, return_splits=True)  #512, 288 # 352,128 #LGR had 128,128 MobileNetv2 had 224, 224
    domain_dataloaders[domain] = loaders

testing_scenarios = {
    'simple': ('3conv', 'simple', False, 1000), 
    'simple_buffer05': ('3conv', 'simple', False, 500),
    'special': ('3conv', 'special', False, 1000),
    'special_bnorm': ('3conv', 'special', True, 1000)
}

for name, (backbone, branch, batch_norm, buffer_size) in testing_scenarios.items():
    print(f"\nTesting: {backbone=} {branch=} {batch_norm=} {buffer_size=}")
    dual_model = DualBranchNet_minimal(backbone_type=backbone, branch_type=branch, head_type='3layer', batch_norm=batch_norm).to(device)
    optimizer = torch.optim.Adam(dual_model.parameters(), lr=1e-3)
    buffer = NaiveRehearsalBuffer(buffer_size=buffer_size)

    exp_name = f"minimal_{name}_dualbranchmodel_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}"
    writer = SummaryWriter(log_dir=f"../tensorboard/{exp_name}")

    baseline_kwargs = {'mse_criterion': nn.MSELoss()}
    unified_train_loop(
        model=dual_model,
        domains=domains,
        domain_dataloaders=domain_dataloaders,
        buffer=buffer,
        optimizer=optimizer,
        writer=writer,
        device=device,
        batch_fn=baseline_batch,
        batch_kwargs=baseline_kwargs,
        num_epochs=10,
        exp_name=exp_name,
        gradient_clipping=True,
        detach_base=False,
        binary = True,
        full_replay = True,
        collect_tsne_data=False,
        loss_params={}
    )

In [ ]:
import datetime
from torch.utils.tensorboard import SummaryWriter

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
domains = df['domain'].unique()
domain_to_idx = {d: i for i, d in enumerate(domains)}

testing_scenarios = {
    # 'dualbranch_CNN_1epoch': (DualBranchCNNNet('3conv', 'simple', 'simple'), 500, 1), 
    'minimal_absurd_buff500': (DualBranchNet_minimal(backbone_type='absurd', branch_type='absurd', head_type='absurd'), 500, 10),
}

for name, (model, buffer_size, epochs) in testing_scenarios.items():
    print(f"\nTesting: {name}")
    dual_model = model.to(device)
    optimizer = torch.optim.Adam(dual_model.parameters(), lr=1e-3)
    buffer = NaiveRehearsalBuffer(buffer_size=buffer_size)

    exp_name = f"{name}_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}"
    writer = SummaryWriter(log_dir=f"../tensorboard/{exp_name}")

    def cos_criterion(a, b):
        return (1 - torch.abs(nn.CosineSimilarity()(a, b))).mean()
    
    baseline_kwargs = {'mse_criterion': nn.MSELoss()}
    dualbranch_kwargs = {
            'mse_criterion': nn.MSELoss(),
            'ce_criterion': nn.CrossEntropyLoss(),
            'cos_criterion': cos_criterion,
            'domain_to_idx': domain_to_idx,
            'bce_criterion': nn.BCEWithLogitsLoss()
        }
    
    unified_train_loop(
        model=dual_model,
        domains=domains,
        domain_dataloaders=domain_dataloaders,
        buffer=buffer,
        optimizer=optimizer,
        writer=writer,
        device=device,
        batch_fn=baseline_batch,
        batch_kwargs=dualbranch_kwargs,
        num_epochs=epochs,
        exp_name=exp_name,
        gradient_clipping=True,
        detach_base=False,
        binary = True,
        full_replay = True,
        collect_tsne_data=False,
        loss_params={}
    )

In [ ]:
import datetime
from torch.utils.tensorboard import SummaryWriter

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
domains = df['domain'].unique()
domain_to_idx = {d: i for i, d in enumerate(domains)}

testing_scenarios = {
    'DANN_notrain1dom': (DualBranchNet_DANN(), 500, 10, 1)
}

for name, (model, buffer_size, epochs, alpha) in testing_scenarios.items():
    print(f"\nTesting: {name}")
    dual_model = model.to(device)
    optimizer = torch.optim.Adam(dual_model.parameters(), lr=1e-3)
    buffer = NaiveRehearsalBuffer(buffer_size=buffer_size)

    exp_name = f"{name}_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}"
    writer = None #SummaryWriter(log_dir=f"../tensorboard/{exp_name}")

    def cos_criterion(a, b):
        return (1 - torch.abs(nn.CosineSimilarity()(a, b))).mean()
    
    dualbranch_kwargs = {
            'mse_criterion': nn.MSELoss(),
            'ce_criterion': nn.CrossEntropyLoss(),
            'cos_criterion': cos_criterion,
            'domain_to_idx': domain_to_idx,
            'bce_criterion': nn.BCEWithLogitsLoss(),
            'alpha': alpha,
        }
    
    unified_train_loop(
        model=dual_model,
        domains=domains,
        domain_dataloaders=domain_dataloaders,
        buffer=buffer,
        optimizer=optimizer,
        writer=writer,
        device=device,
        batch_fn=dann_batch,
        batch_kwargs=dualbranch_kwargs,
        num_epochs=epochs,
        exp_name=exp_name,
        gradient_clipping=True,
        detach_base=False,
        binary = True,
        full_replay = True,
        collect_tsne_data=False,
        loss_params={'head':0.5, 'social':0.5}
    )

testing_scenarios = {
    'dualbranch_CNN_1epoch': (DualBranchCNNNet(backbone_type='3conv', branch_type='simple', end_type='3linear'), 500, 1, 1), 
    'dualbranch_CNN_dynamicalpha': (DualBranchCNNNet(backbone_type='3conv', branch_type='simple', end_type='3linear'), 500, 10, 0), 
}

for name, (model, buffer_size, epochs, alpha) in testing_scenarios.items():
    print(f"\nTesting: {name}")
    dual_model = model.to(device)
    optimizer = torch.optim.Adam(dual_model.parameters(), lr=1e-3)
    buffer = NaiveRehearsalBuffer(buffer_size=buffer_size)

    exp_name = f"{name}_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}"
    writer = None #SummaryWriter(log_dir=f"../tensorboard/{exp_name}")

    def cos_criterion(a, b):
        return (1 - torch.abs(nn.CosineSimilarity()(a, b))).mean()
    
    dualbranch_kwargs = {
            'mse_criterion': nn.MSELoss(),
            'ce_criterion': nn.CrossEntropyLoss(),
            'cos_criterion': cos_criterion,
            'domain_to_idx': domain_to_idx,
            'bce_criterion': nn.BCEWithLogitsLoss(),
            'alpha': alpha,
        }
    
    unified_train_loop(
        model=dual_model,
        domains=domains,
        domain_dataloaders=domain_dataloaders,
        buffer=buffer,
        optimizer=optimizer,
        writer=writer,
        device=device,
        batch_fn=dualbranch_batch,
        batch_kwargs=dualbranch_kwargs,
        num_epochs=epochs,
        exp_name=exp_name,
        gradient_clipping=True,
        detach_base=False,
        binary = True,
        full_replay = True,
        collect_tsne_data=False,
        loss_params={'head':1, 'social':1, 'room':1}
    )

In [ ]:
import datetime
from torch.utils.tensorboard import SummaryWriter

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
domains = df['domain'].unique()
domain_to_idx = {d: i for i, d in enumerate(domains)}

testing_scenarios = {
    '3conv_simple_simple_bnorm_nogradclip': (DualBranchCNNNet(backbone_type='3conv', branch_type='simple', end_type='simple', batch_norm=True), 500, 10, 1, False), 
    '3conv_simple_simple_nogradclip': (DualBranchCNNNet(backbone_type='3conv', branch_type='simple', end_type='simple', batch_norm=False), 500, 10, 1, False), 
    '3conv_simple_simple_bnorm': (DualBranchCNNNet(backbone_type='3conv', branch_type='simple', end_type='simple', batch_norm=True), 500, 10, 1, True),
    '3conv_simple_3linear_bnorm_nogradclip': (DualBranchCNNNet(backbone_type='3conv', branch_type='simple', end_type='simple', batch_norm=True), 500, 10, 1, False),
}

for name, (model, buffer_size, epochs, alpha, gradient_clip) in testing_scenarios.items():
    print(f"\nTesting: {name}")
    dual_model = model.to(device)
    optimizer = torch.optim.Adam(dual_model.parameters(), lr=1e-3)
    buffer = NaiveRehearsalBuffer(buffer_size=buffer_size)

    exp_name = f"{name}_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}"
    writer = None #SummaryWriter(log_dir=f"../tensorboard/{exp_name}")

    def cos_criterion(a, b):
        return (1 - torch.abs(nn.CosineSimilarity()(a, b))).mean()
    
    dualbranch_kwargs = {
            'mse_criterion': nn.MSELoss(),
            'ce_criterion': nn.CrossEntropyLoss(),
            'cos_criterion': cos_criterion,
            'domain_to_idx': domain_to_idx,
            'bce_criterion': nn.BCEWithLogitsLoss(),
            'alpha': alpha,
        }
    
    unified_train_loop(
        model=dual_model,
        domains=domains,
        domain_dataloaders=domain_dataloaders,
        buffer=buffer,
        optimizer=optimizer,
        writer=writer,
        device=device,
        batch_fn=dualbranch_batch,
        batch_kwargs=dualbranch_kwargs,
        num_epochs=epochs,
        exp_name=exp_name,
        gradient_clipping=gradient_clip,
        detach_base=False,
        binary = True,
        full_replay = True,
        collect_tsne_data=False,
        loss_params={'head':1, 'social':1, 'room':1}
    )

In [ ]:
import datetime
from torch.utils.tensorboard import SummaryWriter

df = pd.read_pickle("../data/pepper_data.pkl")
domain_dataloaders = get_dataloader(df, batch_sizes=(32, 64, 64), resize_img_to=(224,224), return_splits=False, double_img=False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
domains = df['domain'].unique()
domain_to_idx = {d: i for i, d in enumerate(domains)}

testing_scenarios = {
    'deeplabv3mobilenetv3_dann': (DANN_classifier_poc(), 500, 30, 0, True),
}

for name, (model, buffer_size, epochs, alpha, gradient_clip) in testing_scenarios.items():
    print(f"\nTesting: {name}")
    dual_model = model.to(device)
    optimizer = torch.optim.Adam(dual_model.parameters(), lr=1e-3)
    buffer = NaiveRehearsalBuffer(buffer_size=buffer_size)

    exp_name = f"{name}_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}"
    writer = None #SummaryWriter(log_dir=f"../tensorboard/{exp_name}")

    def cos_criterion(a, b):
        return (1 - torch.abs(nn.CosineSimilarity()(a, b))).mean()
    
    dualbranch_kwargs = {
            'mse_criterion': nn.MSELoss(),
            'ce_criterion': nn.CrossEntropyLoss(),
            'cos_criterion': cos_criterion,
            'domain_to_idx': domain_to_idx,
            'bce_criterion': nn.BCEWithLogitsLoss(),
            'alpha': alpha,
        }
    
    unified_train_loop(
        model=dual_model,
        domains=domains,
        domain_dataloaders=domain_dataloaders,
        buffer=buffer,
        optimizer=optimizer,
        writer=writer,
        device=device,
        batch_fn=dann_batch,
        batch_kwargs=dualbranch_kwargs,
        num_epochs=epochs,
        exp_name=exp_name,
        gradient_clipping=gradient_clip,
        detach_base=False,
        binary = True,
        full_replay = True,
        collect_tsne_data=True,
        loss_params={'head':1, 'social':1}
    )

In [ ]:
#didit work, the architecture is flawed. Assesing category of features based on catastrophic forgetting makes no sense, 
# the features are not experiencing CF, the weights are. 
# But passing weights to split branches is not justified.

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
domains = df['domain'].unique()
domain_to_idx = {d: i for i, d in enumerate(domains)}

df = pd.read_pickle("../data/pepper_data.pkl")
domain_dataloaders = get_dataloader(df, batch_sizes=(32, 64, 64), resize_img_to=(224,224), return_splits=False, double_img=False)

# CORRECTED: Initialize model with proper parameter names
cf_model = CatastrophicForgettingDisentanglementModel(
    backbone_output_channels=640,
    branch_hidden_channels=256,
    branch_output_channels=128,
    num_outputs=9
).to(device)

# CORRECTED: Optimizers with proper parameter paths
main_optimizer = torch.optim.Adam([
    {'params': cf_model.forgetting_adapter.adapter.parameters()},
    {'params': cf_model.branch_forgetting.parameters()},
    {'params': cf_model.branch_not_forgetting.parameters()},
    {'params': cf_model.head.parameters()},
    {'params': cf_model.fusion.parameters()}
], lr=1e-3)

gating_optimizer = torch.optim.Adam(
    cf_model.forgetting_adapter.gating.parameters(),
    lr=1e-4
)

# restart = {
# 'global_step':
# 'history':
# 'domain':
# }

# Use your existing buffer and data
buffer = NaiveRehearsalBuffer(buffer_size=1000)

# Modified batch kwargs for our model
cf_kwargs = {
    'mse_criterion': nn.MSELoss(),
    'domain_to_idx': domain_to_idx,
}

# Run training with corrected functions
exp_name = f"forgetinggated_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}"

history = catastrophic_forgetting_train_loop(
    model=cf_model,
    domains=domains,
    domain_dataloaders=domain_dataloaders,
    buffer=buffer,
    optimizer=main_optimizer,
    gating_optimizer=gating_optimizer,
    device=device,
    batch_fn=catastrophic_forgetting_batch,
    batch_kwargs=cf_kwargs,
    num_epochs=10,
    exp_name=exp_name,
    gradient_clipping=True,
    loss_params={'main': 1.0, 'adapter': 0.5, 'gating': 0.1},
    restart = restart
)

In [ ]:
import datetime

df = pd.read_pickle("../data/pepper_data.pkl")
domain_dataloaders = get_dataloader(df, batch_sizes=(32, 64, 64), resize_img_to=(288,512), return_splits=False, double_img=False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
domains = df['domain'].unique()
domain_to_idx = {d: i for i, d in enumerate(domains)}

testing_scenarios = {
    # 'dann_mobilenet': (DualBranchNet_DANN(backbone_type='mobilenet'), 500, 10, 0, False, dann_batch),
    # 'dual_2conv_adversarial_3linear_detach': (DualBranchCNNNet(9,6,'2conv', 'adversarial', '3linear'), 500, 10, 0, True, dualbranch_batch),
    # 'dual_mobilenet_simple_3linear': (DualBranchCNNNet(9,6,'pretrained', 'simple', '3linear'), 500, 10, 0, False, dualbranch_batch),
    # 'dual_mobilenet_linear_simple': (DualBranchCNNNet(9,6,'pretrained', 'linear', 'simple'), 500, 10, 0, False, dualbranch_batch),
    'baseline': (LGRBaseline(), 500, 10, 0, False, baseline_batch)
}

for name, (model, buffer_size, epochs, alpha, detach_base, batch_fn) in testing_scenarios.items():
    print(f"\nTesting: {name}")
    dual_model = model.to(device)
    optimizer = torch.optim.Adam(dual_model.parameters(), lr=1e-3)
    buffer = NaiveRehearsalBuffer(buffer_size=buffer_size)

    exp_name = f"{name}_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}"
    writer = None

    def cos_criterion(a, b):
        return (1 - torch.abs(nn.CosineSimilarity()(a, b))).mean()
    
    dualbranch_kwargs = {
            'mse_criterion': nn.MSELoss(),
            'ce_criterion': nn.CrossEntropyLoss(),
            'cos_criterion': cos_criterion,
            'domain_to_idx': domain_to_idx,
            'bce_criterion': nn.BCEWithLogitsLoss(),
            'alpha': alpha,
        }
    
    unified_train_loop(
        model=dual_model,
        domains=domains,
        domain_dataloaders=domain_dataloaders,
        buffer=buffer,
        optimizer=optimizer,
        writer=writer,
        device=device,
        batch_fn=batch_fn,
        batch_kwargs=dualbranch_kwargs,
        num_epochs=epochs,
        exp_name=exp_name,
        gradient_clipping=True,
        detach_base=detach_base,
        binary = True,
        full_replay = True,
        collect_tsne_data=False,
        loss_params={'head': 1, 'social': 1, 'room': 0.5}
    )

In [ ]:
df = pd.read_pickle("../data/pepper_data.pkl")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
domains = df['domain'].unique()
domain_to_idx = {d: i for i, d in enumerate(domains)}

domain_dataloaders = get_dataloader(df, batch_sizes=(32, 64, 64), resize_img_to=(288,512), return_splits=False, double_img=False)

testing_scenarios = {
    # 'dann_mobilenet_buff500': (DualBranchNet_DANN(backbone_type='mobilenet'), 500, 10, 0, False, dann_batch),
    'dann_mobilenet_buff120': (DualBranchNet_DANN(backbone_type='mobilenet'), 120, 10, 0, False, dann_batch),
    # 'dual_2conv_adversarial_3linear_detach_buff500': (DualBranchCNNNet(9,6,'2conv', 'adversarial', '3linear'), 500, 10, 0, True, dualbranch_batch),
    # 'dual_2conv_adversarial_3linear_detach_buff120': (DualBranchCNNNet(9,6,'2conv', 'adversarial', '3linear'), 120, 10, 0, True, dualbranch_batch),
    # 'dual_mobilenet_simple_3linear_buff500': (DualBranchCNNNet(9,6,'pretrained', 'simple', '3linear'), 500, 10, 0, False, dualbranch_batch),
    # 'dual_mobilenet_simple_3linear_buff120': (DualBranchCNNNet(9,6,'pretrained', 'simple', '3linear'), 120, 10, 0, False, dualbranch_batch),
    # 'dual_mobilenet_linear_simple_buff120': (DualBranchCNNNet(9,6,'pretrained', 'linear', 'simple'), 120, 10, 0, False, dualbranch_batch),
    # 'baseline_buff500': (LGRBaseline(), 500, 10, 0, False, baseline_batch),
    # 'baseline_buff120': (LGRBaseline(), 120, 10, 0, False, baseline_batch)
}

for name, (model, buffer_size, epochs, alpha, detach_base, batch_fn) in testing_scenarios.items():
    print(f"\nTesting: {name}")
    dual_model = model.to(device)
    optimizer = torch.optim.Adam(dual_model.parameters(), lr=1e-3)
    buffer = NaiveRehearsalBuffer(buffer_size=buffer_size)

    exp_name = f"{name}_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}"
    writer = None

    def cos_criterion(a, b):
        return (1 - torch.abs(nn.CosineSimilarity()(a, b))).mean()
    
    dualbranch_kwargs = {
            'mse_criterion': nn.MSELoss(),
            'ce_criterion': nn.CrossEntropyLoss(),
            'cos_criterion': cos_criterion,
            'domain_to_idx': domain_to_idx,
            'bce_criterion': nn.BCEWithLogitsLoss(),
            'alpha': alpha,
        }
    
    unified_train_loop(
        model=dual_model,
        domains=domains,
        domain_dataloaders=domain_dataloaders,
        buffer=buffer,
        optimizer=optimizer,
        writer=writer,
        device=device,
        batch_fn=batch_fn,
        batch_kwargs=dualbranch_kwargs,
        num_epochs=epochs,
        exp_name=exp_name,
        gradient_clipping=True,
        detach_base=detach_base,
        binary = True,
        full_replay = True,
        collect_tsne_data=False,
        loss_params={'head': 1, 'social': 1, 'room': 0.5}
    )

# import subprocess

# model_names = [
#     'dann_mobilenet_buff500',
#     'dann_mobilenet_buff120',
#     'dual_2conv_adversarial_3linear_detach_buff500',
#     'dual_2conv_adversarial_3linear_detach_buff120',
#     'dual_mobilenet_simple_3linear_buff500',
#     'dual_mobilenet_simple_3linear_buff120',
#     'dual_mobilenet_linear_simple_buff120',
#     'baseline_buff500',
#     'baseline_buff120'
#  ]

# for model_name in model_names:
#     print(f"Running {model_name}")
#     result = subprocess.run(
#         ["python", "train_models.py", "--model_name", model_name],
#         capture_output=True, text=True
#     )
#     print(result.stdout)
#     if result.stderr:
#         print("Error:", result.stderr)


### current

In [ ]:
# TODO tests: retrain the model with
# 3 trains averaged
# one branch, no mask
# top branch + mask
# bottom branch + mask
# both branches + random masks

In [ ]:

testing_scenarios = {
    # 'heuristic_small_env': (DualBranchModel(), [(288,512), (144,256)], False),
    # 'heuristic_square_img': (DualBranchModel(), [(224,224)]*2, False),
    'heuristic_eval_buffer': (DualBranchModel(), [(288,512)]*2, True)
}

for name, (model, img_size, eval_buffer) in testing_scenarios.items():

    transform_soc = transforms.Compose([
        transforms.Resize(img_size[0]),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                            std=[0.229, 0.224, 0.225])
    ])
    transform_env = transforms.Compose([
        transforms.Resize(img_size[1]),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                            std=[0.229, 0.224, 0.225])
    ])

    df = pd.read_pickle("../data/pepper_data.pkl")
    df['image_path_env'] = df['image_path'].apply(lambda p: str(Path('../data/masked/environment') / Path(p).name))
    df['image_path_social'] = df['image_path'].apply(lambda p: str(Path('../data/masked/social') / Path(p).name))
    domain_dataloaders = get_dataloader(df, batch_sizes=(16, 64, 64), return_splits=False, double_img=True, transforms=[transform_soc, transform_env], num_workers=0)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    domains = df['domain'].unique()
    domain_to_idx = {d: i for i, d in enumerate(domains)}


    print(f"\nTesting: {name}")
    dual_model = model.to(device)
    # optimizer = torch.optim.Adam(dual_model.parameters(), lr=1e-3)
    optimizer = optim.Adam([
        {'params': dual_model.social_branch.parameters()},
        {'params': dual_model.env_branch.parameters()},
        {'params': dual_model.head.parameters()},
    ], lr=1e-3)
    classifier_optimizer = optim.Adam([ 
        {'params': dual_model.social_classifier.parameters()},
        {'params': dual_model.env_classifier.parameters()}
    ], lr=1e-3)

    buffer = NaiveRehearsalBuffer(buffer_size=120)
    if eval_buffer:
        eval_buffer = NaiveRehearsalBuffer(buffer_size=120)

    exp_name = f"{name}_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}"
    writer = None

    def cos_criterion(a, b):
        return (1 - torch.abs(nn.CosineSimilarity()(a, b))).mean()
    
    dualbranch_kwargs = {
            'mse_criterion': nn.MSELoss(),
            'ce_criterion': nn.CrossEntropyLoss(),
            'cos_criterion': cos_criterion,
            'domain_to_idx': domain_to_idx,
            'bce_criterion': nn.BCEWithLogitsLoss(),
            'alpha': 0,
            'class_optimizer': classifier_optimizer
        }
    
    unified_train_loop(
        model=dual_model,
        domains=domains,
        domain_dataloaders=domain_dataloaders,
        buffer=buffer,
        optimizer=optimizer,
        writer=writer,
        device=device,
        batch_fn=heuristic_dualbranch_batch,
        batch_kwargs=dualbranch_kwargs,
        num_epochs=10,
        exp_name=exp_name,
        gradient_clipping=True,
        detach_base=False,
        binary = True,
        full_replay = True,
        collect_tsne_data=False,
        loss_params={'head': 1, 'social': 1, 'room': 0.5},
        eval_buffer=eval_buffer
    )

In [ ]:
import subprocess

model_names = [
    'heuristic_small_env',
    'heuristic_square_img',
    'heuristic_eval_buffer'
]

for model_name in model_names:
    print(f"Running {model_name}")
    process = subprocess.Popen(
        ["python", "train_models.py", "--model_name", model_name, "--num_workers", "0"],
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True
    )

    for line in process.stdout:
        print(line, end='')

    process.wait()

In [ ]:
# Ideas for potential ways to combine networks
# residual = SpecificHead(base + invariant_feats)
# specific_feats = invariant_feats + residual

# gate = GateNet(base + invariant_feats)
# residual = SpecificHead(base)
# specific_feats = invariant_feats + gate * residual

# residual = Attention(invariant_feats, base)
# specific_feats = invariant_feats + residual

# gamma, beta = SpecificHead(base)
# specific_feats = gamma * invariant_feats + beta

# invariant_feats = InvariantHead(base)
# specific_feats = SpecificHead(base)
# cos_sim(invariant_feats, specific_feats) ~ 0
